# LIL MILAN SETTEMBRE

In [3]:
pip install -r requirements.txt

   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.7 MB 4.2 MB/s eta 0:00:02
   ----- ---------------------------------- 1.3/8.7 MB 3.5 MB/s eta 0:00:03
   --------- ------------------------------ 2.1/8.7 MB 3.7 MB/s eta 0:00:02
   ------------- -------------------------- 2.9/8.7 MB 3.6 MB/s eta 0:00:02
   --------------- ------------------------ 3.4/8.7 MB 3.5 MB/s eta 0:00:02
   ------------------- -------------------- 4.2/8.7 MB 3.5 MB/s eta 0:00:02
   --------------------- ------------------ 4.7/8.7 MB 3.4 MB/s eta 0:00:02
   -------------------------- ------------- 5.8/8.7 MB 3.5 MB/s eta 0:00:01
   ---------------------------- ----------- 6.3/8.7 MB 3.5 MB/s eta 0:00:01
   ------------------------------- -------- 6.8/8.7 MB 3.4 MB/s eta 0:00:01
   ---------------------------------- ----- 7.6/8.7 MB 3.4 MB/s eta 0:00:01
   ------------------------------------- -- 8.1/8.7 MB 3.3 MB/s eta 0:00:01
   ----------------


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import warnings

from scripts.ordini import Ordini

# from classes import Ordini, ShopifyMatcher, ScalapayMatcher, SatispayMatcher, PaypalMatcher, QromoMatcher, BonificoMatcher, MatcherRunner

pd.set_option('display.max_columns', None)
pd.reset_option('display.max_rows', None)
warnings.filterwarnings("ignore")


filepath = "C:\\Users\\isabe\\Downloads\\"

In [2]:
ordini_processor = Ordini(filepath, mese=10, anno=2024)
ordini, df_columns = ordini_processor.preprocess() 

KeyError: 'Brand'

In [ ]:
def reformat_date(date_str):
    if pd.notna(date_str):
        date_str = date_str.strip().replace("/", "-")[:10]
        # Check if the string starts the year
        if "-" in date_str[:4]:
            # Split and rearrange as "YYYY-MM-DD"
            return "-".join(date_str.split("-")[::-1])
    return date_str  # Return as-is if already starts with "202"


d = reformat_date("20-12-2024")

NameError: name 'pd' is not defined

In [6]:
o = pd.read_csv(filepath+"LIL Milan Orders Export 10.2024.csv") #, thousands='.')
# o[o.columns] = o.groupby('Name')[o.columns].ffill()
# o["Payment Method"] = o["Payment Method"].str.strip()
# o = handle_cambi(o)

# o2 = pd.read_csv(filepath+"Ordini LIL.csv") #, thousands='.')
# o2[o2.columns] = o2.groupby('Name')[o2.columns].ffill()
# # o = o[o["Payment Method"].str.contains("PayPal Express Checkout", na=False)]

In [7]:
o.to_excel("ordini_iniziali.xlsx")

### QROMO

In [46]:
q = pd.read_csv(filepath+"lilmilan_cronologia delle transazioni_05_11_2024, 10_04_19.csv", thousands='.', decimal=",")
q

,Data,Stato,Importo €,Importo rimborsato €,ID
0,2024-11-02 19:11:32,Riuscito,140.0,0,pi_3QGlnML1BrR61arW0MBbM4sO
1,2024-11-02 19:09:32,Riuscito,240.0,0,pi_3QGllQL1BrR61arW16jD2D7d
2,2024-11-02 17:56:03,Riuscito,320.0,0,pi_3QGkcJL1BrR61arW0KCvrrYj
3,2024-11-02 16:29:12,Riuscito,340.0,0,pi_3QGjGGL1BrR61arW0V26nKpt
4,2024-11-02 16:28:21,Riuscito,480.0,0,pi_3QGjFRL1BrR61arW1RGuSLtB
...,...,...,...,...,...
995,2023-08-30 16:02:36,Riuscito,70.0,0,pi_3NkoyeL1BrR61arW07eOjeuD
996,2023-08-30 13:13:34,Riuscito,250.0,0,pi_3NkmL4L1BrR61arW0MmQfAb3
997,2023-08-30 12:08:45,Riuscito,110.0,0,pi_3NklKLL1BrR61arW1Q3sTYZl
998,2023-08-29 18:47:46,Riuscito,360.0,0,pi_3NkV4wL1BrR61arW1veeSZsn


In [74]:
q = q[q["Stato"] != "Annullato"]

q = q[q["Data"].str.startswith("2024-10")]
q["Importo €"] = q["Importo €"].astype(float)
q["Importo rimborsato €"] = q["Importo rimborsato €"].astype(float)
q["Importo Pagato"] = q["Importo €"] - q["Importo rimborsato €"]
q = q[q["Importo Pagato"] != 0]
q['Numero Pagamento'] = q['ID'] 

df = q.copy()

In [33]:
print(len(q), len(o))

122 1414


In [115]:
df['partial_date'] =  pd.to_datetime(df['Data']).dt.tz_localize(None).dt.date
df = df[["Data", "Stato", "Importo €", "Importo rimborsato €", "Importo Pagato", "Numero Pagamento", "partial_date"]]
df = df.sort_values('Data')

o["Payment Method"] = o["Payment Method"].str.replace("Custom (POS)", "Qromo")
o = o[o["Payment Method"].str.contains("Qromo", na=False)]
df_ordini = o[o['Payment Method'].str.contains('Qromo', case=False, na=False)]
paid_at = df_ordini['Paid at'].str.replace(r'\s[+-]\d{4}$', '', regex=True)
df_ordini['partial_date'] = pd.to_datetime(paid_at).dt.tz_localize(None).dt.date
df_ordini = df_ordini.sort_values('Paid at')
df_ordini["Payment Method"] = df_ordini["Payment Method"].str.strip()

In [82]:
def process_check_groups(group):
    if "VERO" in group["CHECK"].values:
        return group[group["CHECK"] == "VERO"]
    else:
        return group

In [76]:
def check_values(row):
    total = row["Total"]
    amount = row["Importo Pagato"]

    if pd.isna(total) or pd.isna(amount):
        return "NON TROVATO"
    elif total == amount:
        return "VERO"
    else:
        return "FALSO"

In [117]:
#nuova logica matching
df_ordini_solo = df_ordini.loc[df_ordini["Payment Method"] == "Qromo"]

#solo falso perchè prendiamo solo gli ordini che hanno Qromo come unico metodo di pagamento 
df_check = pd.merge(df_ordini_solo, df, left_on=['partial_date', "Total"], right_on = ["partial_date", "Importo Pagato"], how='outer', suffixes=('_ordini', '_qromo'))

# create Time_difference
paid_at = pd.to_datetime(df_check['Paid at'].str.replace(r'\s[+-]\d{4}$', '', regex=True)).reset_index(drop=True)
df_check['Data'] = pd.to_datetime(df_check['Data']).reset_index(drop=True)
df_check['Time_difference'] = abs(paid_at.dt.tz_localize(None) - df_check['Data'])

#tieni solo coppia pagamento-ordine più vicina nel tempo
df_check2 = df_check.sort_values("Data")
names = df_check2["Numero Pagamento"].unique()
    
for n in names:
    f = df_check2[(df_check2["Numero Pagamento"] == n)]
    if f['Time_difference'].isna().all():
        continue  # Skip this iteration if all values are NaN
    min_days_idx = f['Time_difference'].idxmin()
    order = f.loc[min_days_idx, "Name"]
    rows_to_drop = df_check2[(df_check2["Name"] == order) & (df_check2["Numero Pagamento"] != n) | (df_check2["Name"] != order) & (df_check2["Numero Pagamento"] == n)]
    df_check2 = df_check2.drop(rows_to_drop.index, axis = 0)

df_check2 = df_check2.drop_duplicates()
df_check2 = df_check2[(~df_check2["Name"].isna()) & (~df_check2["Numero Pagamento"].isna())]

#cambia check a VERO per ordini e pagamenti matchati
nomi_vero = df_check2["Name"].unique()
pagamenti_vero = df_check2["Numero Pagamento"].unique()

#add to df_check Names and Numero Pagamento that we lost in the matching
# missing_names = set(df_ordini.Name.unique()) - set(df_check2.Name.unique())
# missing_num = set(df["Numero Pagamento"].unique()) - set(df_check2["Numero Pagamento"].unique())

# missing_rows_step2 = df_ordini[df_ordini["Name"].isin(missing_names)]
# missing_rows_num = df[df["Numero Pagamento"].isin(missing_num)]
# df_steps = pd.concat([df_check2, missing_rows_step2, missing_rows_num])

# passiamo agli ordini che non hanno solo Qromo come metodo di pagamento
df_mix = df[~df["Numero Pagamento"].isin(pagamenti_vero)]  
df_ordini_suggestions = df_ordini[~df_ordini["Name"].isin(nomi_vero)]

df_check3 = pd.merge(df_ordini_suggestions, df_mix, on= "partial_date", how='outer', suffixes=('_ordini', '_qromo'))
df_check3 = df_check3.drop_duplicates()
df_check3 = df_check3[~df_check3["Name"].isna()]

#calculate time difference
paid_at = pd.to_datetime(df_check3['Paid at'].str.replace(r'\s[+-]\d{4}$', '', regex=True)).dt.tz_localize(None) #.reset_index(drop=True).dt.tz_localize(None) 
df_check3['Data'] = pd.to_datetime(df_check3['Data'])#.reset_index(drop=True)
df_check3['Time_difference'] = abs(paid_at- df_check3['Data'])
df_check3 = df_check3.drop_duplicates()

# df_check4 = df_check3.sort_values("Paid at")
# names = df_check4["Name"].unique()
    
# for n in names:
#     f = df_check4[(df_check4["Name"] == n)]
#     if f['Time_difference'].isna().all():
#         continue  # Skip this iteration if all values are NaN
#     min_days_idx = f['Time_difference'].idxmin()
#     order = f.loc[min_days_idx, "Numero Pagamento"]
#     rows_to_drop = df_check4[(df_check4["Numero Pagamento"] == order) & (df_check4["Name"] != n) | (df_check4["Numero Pagamento"] != order) & (df_check4["Name"] == n)]
#     df_check4 = df_check4.drop(rows_to_drop.index, axis = 0)

# df_check4 = df_check4.drop_duplicates()

df_check3.loc[(df_check3["Total"] == df_check3["Importo Pagato"]), "CHECK"] = "VERO"
df_check3.loc[(~df_check3["Numero Pagamento"].isna()) & (df_check3["Total"] != df_check3["Importo Pagato"]), "CHECK"] = "FALSO"
df_check3.loc[(df_check3["Numero Pagamento"].isna()), "CHECK"] = "NON TROVATO"

# Identify the names that are in df_ordini_solo but not in df_step1
missing_names = set(df_ordini_suggestions.Name.unique()) - set(df_check3.Name.unique())
missing_num = set(df_mix["Numero Pagamento"].unique()) - set(df_check3["Numero Pagamento"].unique())

# Filter df_ordini_suggestions to include only the rows with missing names
missing_rows_step3 = df_ordini_suggestions[df_ordini_suggestions["Name"].isin(missing_names)]
missing_rows_num = df_mix[df_mix["Numero Pagamento"].isin(missing_num)]

df_final_check = pd.concat([df_check2, df_check3, missing_rows_step3, missing_rows_num])
# df_final_check = pd.concat([df_steps, df_check3, missing_rows_step3, missing_rows_num])

In [118]:
df_final_check["CHECK"] = df_final_check.apply(lambda row: check_values(row), axis=1)
df_final_check = df_final_check.groupby("Name", group_keys=False).apply(process_check_groups) 
df_final_check = df_final_check.drop_duplicates()

In [119]:
len(df_ordini.Name.unique())

118

In [120]:
df_final_check[df_final_check["Name"] == "#42538"]
# df_check2[df_check2["Name"] == "#42165"]

# df_check3[df_check3["Name"] == "#42165"]
# df_steps[df_steps["Name"] == "#42165"]

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,partial_date,Data,Stato,Importo €,Importo rimborsato €,Importo Pagato,Numero Pagamento,Time_difference,CHECK
24,#42538,elena.assante@gmail.com,paid,2024-10-24 14:44:45 +0200,fulfilled,2024-10-24 14:44:45 +0200,yes,EUR,45.0,0.0,8.12,45.0,NaN,0.0,NaN,2024-10-24 14:44:44 +0200,1.0,Repair Service LIL House - Saldatura bracciale...,20.0,NaN,1.579000e+10,False,True,fulfilled,elena assante,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Qromo,rRWcDgLy1nxW317tiK3DjkCRv,0.0,LIL Milan,0.0,Veronica Varetta,LIL house,22.0,6.365254e+12,NaN,Low,pos,0.0,IT IVA 22%,8.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2705,NaN,NaN,NaN,rRWcDgLy1nxW317tiK3DjkCRv,NaN,NaN,rRWcDgLy1nxW317tiK3DjkCRv,2024-10-24,2024-10-24 17:43:00,Riuscito,20.0,0.0,20.0,pi_3QDTBgL1BrR61arW1I4fvVUD,0 days 02:58:15,FALSO
23,#42538,elena.assante@gmail.com,paid,2024-10-24 14:44:45 +0200,fulfilled,2024-10-24 14:44:45 +0200,yes,EUR,45.0,0.0,8.12,45.0,NaN,0.0,NaN,2024-10-24 14:44:44 +0200,1.0,Repair Service LIL House - Farfallina,25.0,NaN,1.579000e+10,True,True,fulfilled,elena assante,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Qromo,rRWcDgLy1nxW317tiK3DjkCRv,0.0,LIL Milan,0.0,Veronica Varetta,LIL house,22.0,6.365254e+12,NaN,Low,pos,0.0,IT IVA 22%,8.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2705,NaN,NaN,NaN,rRWcDgLy1nxW317tiK3DjkCRv,NaN,NaN,rRWcDgLy1nxW317tiK3DjkCRv,2024-10-24,2024-10-24 17:43:00,Riuscito,20.0,0.0,20.0,pi_3QDTBgL1BrR61arW1I4fvVUD,0 days 02:58:15,FALSO


In [121]:
df_final_check.CHECK.value_counts()

CHECK
VERO           135
FALSO           23
NON TROVATO      5
Name: count, dtype: int64

In [122]:
len(df_final_check.Name.unique())

118

In [35]:
df_ordini_solo = df_ordini[df_ordini["Payment Method"] == "Qromo"]
df_ordini_mix = df_ordini[(df_ordini["Payment Method"] != "Qromo")]
print(len(df_ordini_mix.Name.unique()), len(df_ordini_solo.Name.unique()))

5 113


In [36]:
df_ordini_mix["Payment Method"].value_counts()

Payment Method
Qromo + manual       5
Gift Card + Qromo    2
Cash + Qromo         1
Name: count, dtype: int64

#### MATCH ANCHE PER TOTALE

In [37]:
df_check = pd.merge(df_ordini_solo, df, left_on=['partial_date', "Total"], right_on = ["partial_date", "Importo Pagato"], how='outer', suffixes=('_qromo', '_ordini'))
df_check[["Name", "partial_date", "Paid at", "Total", "Payment Method", "Importo Pagato", "Numero Pagamento", "Data"]]

,Name,partial_date,Paid at,Total,Payment Method,Importo Pagato,Numero Pagamento,Data
0,#42113,2024-10-01,2024-10-01 18:47:59 +0200,120.0,Qromo,120.0,pi_3Q59EdL1BrR61arW0wsFm270,2024-10-01 18:47:39
1,NaN,2024-10-01,NaN,NaN,NaN,180.0,pi_3Q52rfL1BrR61arW0YFGK03a,2024-10-01 11:59:31
2,#22-2601,2024-10-01,2024-10-01 18:36:14 +0200,320.0,Qromo,320.0,pi_3Q592wL1BrR61arW0q9AAp01,2024-10-01 18:35:34
3,#22-2600,2024-10-01,2024-10-01 17:08:48 +0200,1400.0,Qromo,1400.0,pi_3Q57gVL1BrR61arW17mWj2gF,2024-10-01 17:08:19
4,#42128,2024-10-02,2024-10-02 13:12:01 +0200,20.0,Qromo,20.0,pi_3Q5QT2L1BrR61arW1cvyeAL2,2024-10-02 13:11:40
...,...,...,...,...,...,...,...,...
194,#43003,2024-10-31,2024-10-31 11:10:26 +0100,240.0,Qromo,240.0,pi_3QFxdpL1BrR61arW0nOFe5d8,2024-10-31 13:38:21
195,#22-2754,2024-10-31,2024-10-31 13:38:53 +0100,240.0,Qromo,240.0,pi_3QFvKEL1BrR61arW1cfOOIwX,2024-10-31 11:09:58
196,#22-2754,2024-10-31,2024-10-31 13:38:53 +0100,240.0,Qromo,240.0,pi_3QFxKyL1BrR61arW0HviTjrm,2024-10-31 13:18:52
197,#22-2754,2024-10-31,2024-10-31 13:38:53 +0100,240.0,Qromo,240.0,pi_3QFxdpL1BrR61arW0nOFe5d8,2024-10-31 13:38:21


In [38]:
# Remove timezone from 'Paid at' and convert to datetime
paid_at = pd.to_datetime(df_check['Paid at'].str.replace(r'\s[+-]\d{4}$', '', regex=True)).reset_index(drop=True)

# Convert 'Data' column to datetime
df_check['Data'] = pd.to_datetime(df_check['Data']).reset_index(drop=True)

# Calculate time difference, ignoring time zones
df_check['Time_difference'] = abs(paid_at.dt.tz_localize(None) - df_check['Data'])

df_check[["Name", "Paid at", "Total", "Payment Method", "Importo Pagato", "Numero Pagamento", "Data", "Time_difference"]]

,Name,Paid at,Total,Payment Method,Importo Pagato,Numero Pagamento,Data,Time_difference
0,#42113,2024-10-01 18:47:59 +0200,120.0,Qromo,120.0,pi_3Q59EdL1BrR61arW0wsFm270,2024-10-01 18:47:39,0 days 00:00:20
1,NaN,NaN,NaN,NaN,180.0,pi_3Q52rfL1BrR61arW0YFGK03a,2024-10-01 11:59:31,NaT
2,#22-2601,2024-10-01 18:36:14 +0200,320.0,Qromo,320.0,pi_3Q592wL1BrR61arW0q9AAp01,2024-10-01 18:35:34,0 days 00:00:40
3,#22-2600,2024-10-01 17:08:48 +0200,1400.0,Qromo,1400.0,pi_3Q57gVL1BrR61arW17mWj2gF,2024-10-01 17:08:19,0 days 00:00:29
4,#42128,2024-10-02 13:12:01 +0200,20.0,Qromo,20.0,pi_3Q5QT2L1BrR61arW1cvyeAL2,2024-10-02 13:11:40,0 days 00:00:21
...,...,...,...,...,...,...,...,...
194,#43003,2024-10-31 11:10:26 +0100,240.0,Qromo,240.0,pi_3QFxdpL1BrR61arW0nOFe5d8,2024-10-31 13:38:21,0 days 02:27:55
195,#22-2754,2024-10-31 13:38:53 +0100,240.0,Qromo,240.0,pi_3QFvKEL1BrR61arW1cfOOIwX,2024-10-31 11:09:58,0 days 02:28:55
196,#22-2754,2024-10-31 13:38:53 +0100,240.0,Qromo,240.0,pi_3QFxKyL1BrR61arW0HviTjrm,2024-10-31 13:18:52,0 days 00:20:01
197,#22-2754,2024-10-31 13:38:53 +0100,240.0,Qromo,240.0,pi_3QFxdpL1BrR61arW0nOFe5d8,2024-10-31 13:38:21,0 days 00:00:32


In [39]:
df_step2 = df_check.sort_values("Data")
names = df_step2["Numero Pagamento"].unique()
    
for n in names:
    f = df_step2[(df_step2["Numero Pagamento"] == n)]
    # print(f['Time_difference'].dtype)

    if f['Time_difference'].isna().all():
        # print(f"All 'Time_difference' values are NaN for Numero Pagamento {n}")
        continue  # Skip this iteration if all values are NaN
    min_days_idx = f['Time_difference'].idxmin()
    order = f.loc[min_days_idx, "Name"]
    # order = min_row["Name"]
    rows_to_drop = df_step2[(df_step2["Name"] == order) & (df_step2["Numero Pagamento"] != n) | (df_step2["Name"] != order) & (df_step2["Numero Pagamento"] == n)]
    df_step2 = df_step2.drop(rows_to_drop.index, axis = 0)


df_step2 = df_step2.drop_duplicates(subset=['Name', 'Numero Pagamento'])
df_step2[["Name", "Paid at", "Total", "Payment Method", "Importo Pagato", "Numero Pagamento", "Data", "Time_difference"]]


,Name,Paid at,Total,Payment Method,Importo Pagato,Numero Pagamento,Data,Time_difference
1,NaN,NaN,NaN,NaN,180.0,pi_3Q52rfL1BrR61arW0YFGK03a,2024-10-01 11:59:31,NaT
3,#22-2600,2024-10-01 17:08:48 +0200,1400.0,Qromo,1400.0,pi_3Q57gVL1BrR61arW17mWj2gF,2024-10-01 17:08:19,0 days 00:00:29
2,#22-2601,2024-10-01 18:36:14 +0200,320.0,Qromo,320.0,pi_3Q592wL1BrR61arW0q9AAp01,2024-10-01 18:35:34,0 days 00:00:40
0,#42113,2024-10-01 18:47:59 +0200,120.0,Qromo,120.0,pi_3Q59EdL1BrR61arW0wsFm270,2024-10-01 18:47:39,0 days 00:00:20
4,#42128,2024-10-02 13:12:01 +0200,20.0,Qromo,20.0,pi_3Q5QT2L1BrR61arW1cvyeAL2,2024-10-02 13:11:40,0 days 00:00:21
...,...,...,...,...,...,...,...,...
25,#42165,2024-10-05 12:24:17 +0200,5.0,Qromo,NaN,NaN,NaT,NaT
43,#42244,2024-10-09 10:43:33 +0200,120.0,Qromo,NaN,NaN,NaT,NaT
46,#42263,2024-10-10 11:44:32 +0200,20.0,Qromo,NaN,NaN,NaT,NaT
114,#42526,2024-10-23 18:31:51 +0200,130.0,Qromo,NaN,NaN,NaT,NaT


In [40]:
set(df_ordini_solo.Name.unique()) - set(df_step2.Name.unique())

{'#42538'}

In [41]:
set(df["Numero Pagamento"].unique()) - set(df_step2["Numero Pagamento"].unique())

{'pi_3QFxdpL1BrR61arW0nOFe5d8'}

In [42]:
# Identify the names that are in df_ordini_solo but not in df_step1
missing_names = set(df_ordini_solo.Name.unique()) - set(df_step2.Name.unique())
missing_num = set(df["Numero Pagamento"].unique()) - set(df_step2["Numero Pagamento"].unique())

# Filter df_ordini_solo to include only the rows with missing names
missing_rows_step2 = df_ordini_solo[df_ordini_solo["Name"].isin(missing_names)]
missing_rows_num = df[df["Numero Pagamento"].isin(missing_num)]

In [43]:
df_steps = pd.concat([df_step2, missing_rows_step2, missing_rows_num])
df_steps

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,partial_date,Data,Stato,Importo €,Importo rimborsato €,Importo Pagato,Numero Pagamento,Time_difference
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-01,2024-10-01 11:59:31,Riuscito,180.0,0.0,180.0,pi_3Q52rfL1BrR61arW0YFGK03a,NaT
3,#22-2600,michele.lattanzi@hotmail.com,paid,2024-10-01 17:08:48 +0200,fulfilled,2024-10-01 17:08:48 +0200,no,EUR,1400.0,0.0,252.46,1400.0,NaN,0.0,NaN,2024-10-01 17:08:47 +0200,1.0,B - Yellow / 11 / White Natural Diamond,1400.0,NaN,1.579000e+10,True,True,fulfilled,Michele Lattanzi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Co,NaN,NaN,Qromo,#22-2600.1,0.0,LIL Milan,0.0,Veronica Varetta,LIL house,22.0,6.329802e+12,NaN,Low,pos,0.0,IT IVA 22%,252.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2600,NaN,NaN,NaN,#22-2600.1,NaN,NaN,#22-2600.1,2024-10-01,2024-10-01 17:08:19,Riuscito,1400.0,0.0,1400.0,pi_3Q57gVL1BrR61arW17mWj2gF,0 days 00:00:29
2,#22-2601,sarariga12@gmail.com,paid,2024-10-01 18:36:14 +0200,fulfilled,2024-10-01 18:36:14 +0200,no,EUR,320.0,0.0,57.70,320.0,NaN,0.0,NaN,2024-10-01 18:36:14 +0200,1.0,LIL Brick - Yellow,320.0,NaN,1.579000e+10,True,True,fulfilled,Sara Rigamonti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Qromo,#22-2601.1,0.0,LIL Milan,0.0,Veronica Varetta,LIL house,22.0,6.329938e+12,NaN,Low,pos,0.0,IT IVA 22%,57.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2601,NaN,NaN,NaN,#22-2601.1,NaN,NaN,#22-2601.1,2024-10-01,2024-10-01 18:35:34,Riuscito,320.0,0.0,320.0,pi_3Q592wL1BrR61arW0q9AAp01,0 days 00:00:40
0,#42113,francescapela353@gmail.com,paid,2024-10-01 18:47:59 +0200,fulfilled,2024-10-01 18:47:59 +0200,no,EUR,120.0,0.0,21.64,120.0,NaN,0.0,NaN,2024-10-01 18:47:59 +0200,1.0,Boys Tears Ring - Yellow / 15,120.0,NaN,1.579000e+10,True,True,fulfilled,Francesca Pela,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Qromo,rwVKE549J1oGL6a2K7XAgEUDE,0.0,LIL Milan,0.0,Veronica Varetta,LIL house,22.0,6.329956e+12,NaN,Low,pos,0.0,IT IVA 22%,21.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2602,NaN,NaN,NaN,rwVKE549J1oGL6a2K7XAgEUDE,NaN,NaN,rwVKE549J1oGL6a2K7XAgEUDE,2024-10-01,2024-10-01 18:47:39,Riuscito,120.0,0.0,120.0,pi_3Q59EdL1BrR61arW0wsFm270,0 days 00:00:20
4,#42128,irene.magrelli@hotmail.it,paid,2024-10-02 13:12:01 +0200,fulfilled,2024-10-02 13:12:02 +0200,no,EUR,20.0,0.0,3.61,20.0,NaN,0.0,NaN,2024-10-02 13:12:01 +0200,1.0,Repair Service LIL House - Saldatura collana,20.0,NaN,1.579000e+10,False,True,fulfilled,Irene Magrelli,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Qromo,rOxQGlWmLJaJ4ie3LPL3plJpg,0.0,LIL 

#### MATCH SOLO DATA

In [44]:
df_mix = df_steps[df_steps["Name"].isna()][["partial_date", "Data", "Stato", "Importo €", "Importo rimborsato €", "Importo Pagato",	"Numero Pagamento"]]
df_steps = df_steps[~df_steps["Name"].isna()]
df_mix

,partial_date,Data,Stato,Importo €,Importo rimborsato €,Importo Pagato,Numero Pagamento
1,2024-10-01,2024-10-01 11:59:31,Riuscito,180.0,0.0,180.0,pi_3Q52rfL1BrR61arW0YFGK03a
35,2024-10-05,2024-10-05 12:23:57,Riuscito,365.0,0.0,365.0,pi_3Q6V9VL1BrR61arW1ee3qj1D
28,2024-10-05,2024-10-05 16:53:33,Riuscito,15.0,0.0,15.0,pi_3Q6ZMPL1BrR61arW1HNGWtQx
39,2024-10-08,2024-10-08 14:46:22,Riuscito,400.0,0.0,400.0,pi_3Q7cnyL1BrR61arW0dp4bGtH
40,2024-10-08,2024-10-08 16:06:46,Riuscito,1700.0,0.0,1700.0,pi_3Q7e3mL1BrR61arW0T8COwc0
42,2024-10-09,2024-10-09 11:32:17,Riuscito,40.0,0.0,40.0,pi_3Q7wFhL1BrR61arW1T5CqSdu
44,2024-10-09,2024-10-09 15:39:19,Riuscito,794.0,0.0,794.0,pi_3Q806lL1BrR61arW1x3OjV6Z
63,2024-10-11,2024-10-11 13:27:05,Riuscito,100.0,0.0,100.0,pi_3Q8gztL1BrR61arW1tL4lmCi
62,2024-10-11,2024-10-11 18:23:01,Riuscito,40.0,0.0,40.0,pi_3Q8lcHL1BrR61arW1TRt6nf3
112,2024-10-22,2024-10-22 18:51:03,Riuscito,140.0,0.0,140.0,pi_3QClIRL1BrR61arW0NZm05jX


In [45]:
df_check = pd.merge(df_ordini_mix, df_mix, on= "partial_date", how='outer', suffixes=('_qromo', '_ordini'))
df_check = df_check.drop_duplicates()

non_trovati = df_check[df_check["Name"].isna()][["partial_date", "Data", "Stato", "Importo €", "Importo rimborsato €",	"Importo Pagato",	"Numero Pagamento"]]
# df_check = df_check[~df_check["Name"].isna()]
# df_check = df_check[df_check["Total"] <= df_check["Importo Pagato"]]

df_check[["Name", "partial_date", "Paid at", "Total", "Payment Method", "Importo Pagato", "Numero Pagamento", "Data"]]

,Name,partial_date,Paid at,Total,Payment Method,Importo Pagato,Numero Pagamento,Data
0,NaN,2024-10-01,NaN,NaN,NaN,180.0,pi_3Q52rfL1BrR61arW0YFGK03a,2024-10-01 11:59:31
1,#42174,2024-10-05,2024-10-05 16:53:49 +0200,140.0,Gift Card + Qromo,365.0,pi_3Q6V9VL1BrR61arW1ee3qj1D,2024-10-05 12:23:57
2,#42174,2024-10-05,2024-10-05 16:53:49 +0200,140.0,Gift Card + Qromo,15.0,pi_3Q6ZMPL1BrR61arW1HNGWtQx,2024-10-05 16:53:33
3,#42233,2024-10-08,2024-10-08 14:47:07 +0200,1100.0,Cash + Qromo,400.0,pi_3Q7cnyL1BrR61arW0dp4bGtH,2024-10-08 14:46:22
4,#42233,2024-10-08,2024-10-08 14:47:07 +0200,1100.0,Cash + Qromo,1700.0,pi_3Q7e3mL1BrR61arW0T8COwc0,2024-10-08 16:06:46
5,NaN,2024-10-09,NaN,NaN,NaN,40.0,pi_3Q7wFhL1BrR61arW1T5CqSdu,2024-10-09 11:32:17
6,NaN,2024-10-09,NaN,NaN,NaN,794.0,pi_3Q806lL1BrR61arW1x3OjV6Z,2024-10-09 15:39:19
7,#42311,2024-10-11,2024-10-11 18:35:26 +0200,140.0,Qromo + manual,100.0,pi_3Q8gztL1BrR61arW1tL4lmCi,2024-10-11 13:27:05
8,#42311,2024-10-11,2024-10-11 18:35:26 +0200,140.0,Qromo + manual,40.0,pi_3Q8lcHL1BrR61arW1TRt6nf3,2024-10-11 18:23:01
9,#42311,2024-10-11,2024-10-11 18:35:26 +0200,140.0,Qromo + manual,100.0,pi_3Q8gztL1BrR61arW1tL4lmCi,2024-10-11 13:27:05


In [18]:
# # Remove timezone from 'Paid at' and convert to datetime
df_check['Paid at'] = pd.to_datetime(df_check['Paid at'].str.replace(r'\s[+-]\d{4}$', '', regex=True)).dt.tz_localize(None) #.reset_index(drop=True).dt.tz_localize(None) 

# # Convert 'Data' column to datetime
df_check['Data'] = pd.to_datetime(df_check['Data'])#.reset_index(drop=True)

# # Calculate time difference, ignoring time zones
df_check['Time_difference'] = abs(df_check['Paid at']- df_check['Data'])

df_check = df_check.drop_duplicates(subset=["Name", "Numero Pagamento"])
df_check[["Name", "Paid at", "Total", "Payment Method", "Importo Pagato", "Numero Pagamento", "Data", "Time_difference"]]

,Name,Paid at,Total,Payment Method,Importo Pagato,Numero Pagamento,Data,Time_difference
1,#42174,2024-10-05 16:53:49,140.0,Gift Card + Qromo,365.0,pi_3Q6V9VL1BrR61arW1ee3qj1D,2024-10-05 12:23:57,0 days 04:29:52
2,#42174,2024-10-05 16:53:49,140.0,Gift Card + Qromo,15.0,pi_3Q6ZMPL1BrR61arW1HNGWtQx,2024-10-05 16:53:33,0 days 00:00:16
3,#42233,2024-10-08 14:47:07,1100.0,Cash + Qromo,400.0,pi_3Q7cnyL1BrR61arW0dp4bGtH,2024-10-08 14:46:22,0 days 00:00:45
4,#42233,2024-10-08 14:47:07,1100.0,Cash + Qromo,1700.0,pi_3Q7e3mL1BrR61arW0T8COwc0,2024-10-08 16:06:46,0 days 01:19:39
7,#42311,2024-10-11 18:35:26,140.0,Qromo + manual,100.0,pi_3Q8gztL1BrR61arW1tL4lmCi,2024-10-11 13:27:05,0 days 05:08:21
8,#42311,2024-10-11 18:35:26,140.0,Qromo + manual,40.0,pi_3Q8lcHL1BrR61arW1TRt6nf3,2024-10-11 18:23:01,0 days 00:12:25
11,#42439,2024-10-18 15:52:35,120.0,Gift Card + Qromo,NaN,NaN,NaT,NaT
13,#42497,2024-10-24 17:45:21,160.0,Qromo + manual,20.0,pi_3QDTBgL1BrR61arW1I4fvVUD,2024-10-24 17:43:00,0 days 00:02:21


In [19]:
df_step1 = df_check.sort_values("Paid at")
names = df_step1["Name"].unique()
    
for n in names:
    f = df_step1[(df_step1["Name"] == n)]
    # print(f['Time_difference'].dtype)

    if f['Time_difference'].isna().all():
        # print(f"All 'Time_difference' values are NaN for Numero Pagamento {n}")
        continue  # Skip this iteration if all values are NaN
    min_days_idx = f['Time_difference'].idxmin()
    order = f.loc[min_days_idx, "Numero Pagamento"]
    rows_to_drop = df_step1[(df_step1["Numero Pagamento"] == order) & (df_step1["Name"] != n) | (df_step1["Numero Pagamento"] != order) & (df_step1["Name"] == n)]
    df_step1 = df_step1.drop(rows_to_drop.index, axis = 0)


df_step1 = df_step1.drop_duplicates(subset=['Name', 'Numero Pagamento'])
df_step1[["Name", "Paid at", "Total", "Payment Method", "Importo Pagato", "Numero Pagamento", "Data", "Time_difference"]]

,Name,Paid at,Total,Payment Method,Importo Pagato,Numero Pagamento,Data,Time_difference
2,#42174,2024-10-05 16:53:49,140.0,Gift Card + Qromo,15.0,pi_3Q6ZMPL1BrR61arW1HNGWtQx,2024-10-05 16:53:33,0 days 00:00:16
3,#42233,2024-10-08 14:47:07,1100.0,Cash + Qromo,400.0,pi_3Q7cnyL1BrR61arW0dp4bGtH,2024-10-08 14:46:22,0 days 00:00:45
8,#42311,2024-10-11 18:35:26,140.0,Qromo + manual,40.0,pi_3Q8lcHL1BrR61arW1TRt6nf3,2024-10-11 18:23:01,0 days 00:12:25
11,#42439,2024-10-18 15:52:35,120.0,Gift Card + Qromo,NaN,NaN,NaT,NaT
13,#42497,2024-10-24 17:45:21,160.0,Qromo + manual,20.0,pi_3QDTBgL1BrR61arW1I4fvVUD,2024-10-24 17:43:00,0 days 00:02:21


In [20]:
set(df_ordini_mix.Name.unique()) - set(df_step1.Name.unique())

set()

In [21]:
set(df_mix["Numero Pagamento"].unique()) - set(df_step1["Numero Pagamento"].unique())

{'pi_3Q52rfL1BrR61arW0YFGK03a',
 'pi_3Q6V9VL1BrR61arW1ee3qj1D',
 'pi_3Q7e3mL1BrR61arW0T8COwc0',
 'pi_3Q7wFhL1BrR61arW1T5CqSdu',
 'pi_3Q806lL1BrR61arW1x3OjV6Z',
 'pi_3Q8gztL1BrR61arW1tL4lmCi',
 'pi_3QClIRL1BrR61arW0NZm05jX',
 'pi_3QED2VL1BrR61arW1crlrSQV',
 'pi_3QFdLWL1BrR61arW1Z5jIUcI',
 'pi_3QFdLvL1BrR61arW0KvFDID3',
 'pi_3QFxdpL1BrR61arW0nOFe5d8'}

In [22]:
# Identify the names that are in df_ordini_solo but not in df_step1
missing_names = set(df_ordini_mix.Name.unique()) - set(df_step1.Name.unique())
missing_num = set(df_mix["Numero Pagamento"].unique()) - set(df_step1["Numero Pagamento"].unique())

# Filter df_ordini_solo to include only the rows with missing names
missing_rows_step1 = df_ordini_mix[df_ordini_mix["Name"].isin(missing_names)]
missing_rows_num = df_mix[df_mix["Numero Pagamento"].isin(missing_num)]

In [23]:
df_final = pd.concat([df_steps, df_step1, missing_rows_step1, missing_rows_num])

In [428]:
df_final = df_final.drop_duplicates(subset=["Name", "Numero Pagamento"])

In [24]:
df_final.loc[df_final["Total"] == df_final["Importo Pagato"], "CHECK"] = "VERO"
df_final.loc[df_final["Total"] != df_final["Importo Pagato"], "CHECK"] = "FALSO"
df_final.loc[df_final["Name"].isna() | df_final["Numero Pagamento"].isna(), "CHECK"] = "NON TROVATO"

df_final["CHECK"].value_counts()

CHECK
VERO           107
NON TROVATO     19
FALSO            4
Name: count, dtype: int64

In [25]:
df_final[df_final["CHECK"] == "FALSO"][["Name", "Paid at", "Total", "Payment Method", "Importo Pagato", "Numero Pagamento", "Data", "Time_difference"]]

,Name,Paid at,Total,Payment Method,Importo Pagato,Numero Pagamento,Data,Time_difference
2,#42174,2024-10-05 16:53:49,140.0,Gift Card + Qromo,15.0,pi_3Q6ZMPL1BrR61arW1HNGWtQx,2024-10-05 16:53:33,0 days 00:00:16
3,#42233,2024-10-08 14:47:07,1100.0,Cash + Qromo,400.0,pi_3Q7cnyL1BrR61arW0dp4bGtH,2024-10-08 14:46:22,0 days 00:00:45
8,#42311,2024-10-11 18:35:26,140.0,Qromo + manual,40.0,pi_3Q8lcHL1BrR61arW1TRt6nf3,2024-10-11 18:23:01,0 days 00:12:25
13,#42497,2024-10-24 17:45:21,160.0,Qromo + manual,20.0,pi_3QDTBgL1BrR61arW1I4fvVUD,2024-10-24 17:43:00,0 days 00:02:21


In [26]:
df_final[df_final["CHECK"] == "NON TROVATO"]

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,partial_date,Data,Stato,Importo €,Importo rimborsato €,Importo Pagato,Numero Pagamento,Time_difference,CHECK
25,#42165,pietrogrigolo@gmail.com,paid,2024-10-05 12:24:17 +0200,fulfilled,2024-10-23 15:23:09 +0200,no,EUR,365.0,0.0,65.82,5.0,NaN,400.0,NaN,2024-10-05 12:24:17 +0200,0.0,Girls Tears Necklace - Yellow / 35cm,360.0,NaN,1.579000e+10,True,True,pending,Pietro Grigolo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Qromo,reUdxgh25VLSxfKF1p5gGUp5H,0.0,LIL Milan,0.0,Veronica Varetta,LIL house,22.0,6.335479e+12,NaN,Low,pos,0.0,IT IVA 22%,65.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2617,NaN,NaN,NaN,reUdxgh25VLSxfKF1p5gGUp5H,NaN,NaN,reUdxgh25VLSxfKF1p5gGUp5H,2024-10-05,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NON TROVATO
43,#42244,NaN,paid,2024-10-09 10:43:33 +0200,fulfilled,2024-10-09 10:43:34 +0200,no,EUR,120.0,0.0,21.64,120.0,NaN,0.0,NaN,2024-10-09 10:43:32 +0200,1.0,Say My Name Piercing - Yellow / X / None,120.0,150.0,1.579000e+10,True,True,fulfilled,carlotta trentin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Qromo,r9aaZrala8SkQCqtQaS0Qjmkd,0.0,LIL Milan,0.0,Carlotta Trentin,Firgun House,24.0,6.341405e+12,NaN,Low,pos,0.0,IT IVA 22%,21.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24-1069,NaN,NaN,NaN,r9aaZrala8SkQCqtQaS0Qjmkd,NaN,NaN,r9aaZrala8SkQCqtQaS0Qjmkd,2024-10-09,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NON TROVATO
46,#42263,manuela.facchinetti@fastwebnet.it,paid,2024-10-10 11:44:32 +0200,fulfilled,2024-10-10 11:44:33 +0200,no,EUR,20.0,0.0,3.61,20.0,NaN,0.0,NaN,2024-10-10 11:44:32 +0200,1.0,Repair Service LIL House - Saldatura bracciale...,20.0,NaN,1.579000e+10,False,True,fulfilled,Manuela Facchinetti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Qromo,rPktYjJvruE1naFEzyoV3xKir,0.0,LIL Milan,0.0,Veronica Varetta,LIL house,22.0,6.342991e+12,NaN,Low,pos,0.0,IT IVA 22%,3.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2633,NaN,NaN,NaN,rPktYjJvruE1naFEzyoV3xKir,NaN,NaN,rPktYjJvruE1naFEzyoV3xKir,2024-10-10,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NON TROVATO
114,#42526,NaN,paid,2024-10-23 18:31:51 +0200,fulfilled,2024-10-30 19:10:03 +0100,no,EUR,250.0,0.0,45.08,130.0,NaN,0.0,NaN,2024-10-23 18:31:51 +0200,0.0,Pensavo fosse amore - Yellow / E,120.0,NaN,1.579000e+10,True,True,pending,Marco La Mantina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Qromo,rUglTyUvc8NHQdYS3yJgJF1ZH,0.0,LIL Milan,120.0,Veronica Varetta,LIL house,22.0,6.363635e+12,NaN,Low,pos,0.0,IT IVA 22%,45.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.936634e+11,22-2700,NaN,NaN,NaN,rUglTyUvc8NHQdYS3yJgJF1ZH,NaN,NaN,rUglTyUvc8NHQdYS3yJgJF1ZH,2024-10-23,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NON TROVATO
186,#43006,gebrayeltony@gmail.com,paid,2024-10-31 13:19:14 +0100,fulfilled,2024-11-01 18:21:04 +0100,no,EUR,420.0,0.0,75.74,180.0,NaN,0.0,NaN,2024-10-31 13:19:14 +0100,1.0,Primo bacio Necklace - Yell

### SATISPAY

In [293]:
s = pd.read_csv(filepath+"Satispay LIL House 10.2024.csv") 
s

,payment_uid,payment_date,shop_name,cost_centre,total_amount,fee_amount,payment_type,description,source_payment_uid,satispay_amount,fringe_amount
0,82862003-8ce7-42d5-8f31-8e0d3ec7c43e,2024-10-24 14:44:35.814,lil milan,0,45.0,0.2,TO_BUSINESS,0,0,45.0,0.0
1,e6082596-5ca6-4114-876f-937367fdf1f4,2024-10-23 18:31:32.479,lil milan,0,130.0,0.2,TO_BUSINESS,0,0,130.0,0.0
2,f4924aa1-79ec-4e55-980f-be0a32016c38,2024-10-18 15:52:29.093,lil milan,0,20.0,0.2,TO_BUSINESS,0,0,20.0,0.0
3,d2a681ea-d9c2-484f-8546-a37483dc93b5,2024-10-10 11:44:23.805,lil milan,0,20.0,0.2,TO_BUSINESS,0,0,20.0,0.0


In [314]:
df = s[['payment_uid', 'payment_date', 'total_amount', 'description']]
df['partial_date'] = pd.to_datetime(df['payment_date']).dt.tz_localize(None).dt.date
df = df.rename(columns={"payment_uid": "Numero Pagamento", "payment_date": "Data", "total_amount": "Importo Pagato"})

In [315]:
df_ordini = df_final[df_final["Numero Pagamento"].isna()].loc[:, : "partial_date"]

df_ordini_solo = df_ordini[df_ordini["Payment Method"] == "Qromo"]
df_ordini_mix = df_ordini[(df_ordini["Payment Method"] != "Qromo")]
print(len(df_ordini_mix.Name.unique()), len(df_ordini_solo.Name.unique()))

1 6


#### MATCH ANCHE PER TOTALE

In [299]:
df

,Data,Importo Pagato,Numero Pagamento,partial_date
0,2024-10-24 14:44:35.814,45.0,0,2024-10-24
1,2024-10-23 18:31:32.479,130.0,0,2024-10-23
2,2024-10-18 15:52:29.093,20.0,0,2024-10-18
3,2024-10-10 11:44:23.805,20.0,0,2024-10-10


In [316]:
df_check = pd.merge(df_ordini_solo, df, left_on=['partial_date', "Total"], right_on = ["partial_date", "Importo Pagato"], how='outer', suffixes=('_qromo', '_ordini'))
df_check[["Name", "partial_date", "Paid at", "Total", "Payment Method", "Importo Pagato", "Numero Pagamento", "Data"]]

,Name,partial_date,Paid at,Total,Payment Method,Importo Pagato,Numero Pagamento,Data
0,#42165,2024-10-05,2024-10-05 12:24:17 +0200,5.0,Qromo,NaN,NaN,NaN
1,#42244,2024-10-09,2024-10-09 10:43:33 +0200,120.0,Qromo,NaN,NaN,NaN
2,#42263,2024-10-10,2024-10-10 11:44:32 +0200,20.0,Qromo,20.0,d2a681ea-d9c2-484f-8546-a37483dc93b5,2024-10-10 11:44:23.805
3,NaN,2024-10-18,NaN,NaN,NaN,20.0,f4924aa1-79ec-4e55-980f-be0a32016c38,2024-10-18 15:52:29.093
4,#42526,2024-10-23,2024-10-23 18:31:51 +0200,130.0,Qromo,130.0,e6082596-5ca6-4114-876f-937367fdf1f4,2024-10-23 18:31:32.479
5,#42538,2024-10-24,2024-10-24 14:44:45 +0200,45.0,Qromo,45.0,82862003-8ce7-42d5-8f31-8e0d3ec7c43e,2024-10-24 14:44:35.814
6,#43006,2024-10-31,2024-10-31 13:19:14 +0100,180.0,Qromo,NaN,NaN,NaN


In [317]:
# Remove timezone from 'Paid at' and convert to datetime
paid_at = pd.to_datetime(df_check['Paid at'].str.replace(r'\s[+-]\d{4}$', '', regex=True)).reset_index(drop=True)

# Convert 'Data' column to datetime
df_check['Data'] = pd.to_datetime(df_check['Data']).reset_index(drop=True)

# Calculate time difference, ignoring time zones
df_check['Time_difference'] = abs(paid_at.dt.tz_localize(None) - df_check['Data'])

df_check[["Name", "Paid at", "Total", "Payment Method", "Importo Pagato", "Numero Pagamento", "Data", "Time_difference"]]

,Name,Paid at,Total,Payment Method,Importo Pagato,Numero Pagamento,Data,Time_difference
0,#42165,2024-10-05 12:24:17 +0200,5.0,Qromo,NaN,NaN,NaT,NaT
1,#42244,2024-10-09 10:43:33 +0200,120.0,Qromo,NaN,NaN,NaT,NaT
2,#42263,2024-10-10 11:44:32 +0200,20.0,Qromo,20.0,d2a681ea-d9c2-484f-8546-a37483dc93b5,2024-10-10 11:44:23.805,0 days 00:00:08.195000
3,NaN,NaN,NaN,NaN,20.0,f4924aa1-79ec-4e55-980f-be0a32016c38,2024-10-18 15:52:29.093,NaT
4,#42526,2024-10-23 18:31:51 +0200,130.0,Qromo,130.0,e6082596-5ca6-4114-876f-937367fdf1f4,2024-10-23 18:31:32.479,0 days 00:00:18.521000
5,#42538,2024-10-24 14:44:45 +0200,45.0,Qromo,45.0,82862003-8ce7-42d5-8f31-8e0d3ec7c43e,2024-10-24 14:44:35.814,0 days 00:00:09.186000
6,#43006,2024-10-31 13:19:14 +0100,180.0,Qromo,NaN,NaN,NaT,NaT


In [318]:
df_step2 = df_check.sort_values("Data")
names = df_step2["Numero Pagamento"].unique()
    
for n in names:
    f = df_step2[(df_step2["Numero Pagamento"] == n)]
    # print(f['Time_difference'].dtype)

    if f['Time_difference'].isna().all():
        # print(f"All 'Time_difference' values are NaN for Numero Pagamento {n}")
        continue  # Skip this iteration if all values are NaN
    min_days_idx = f['Time_difference'].idxmin()
    if f.loc[min_days_idx, "Importo Pagato"] != f.loc[min_days_idx, "Total"]:
        continue
    order = f.loc[min_days_idx, "Name"]
    # order = min_row["Name"]
    rows_to_drop = df_step2[(df_step2["Name"] == order) & (df_step2["Numero Pagamento"] != n) | (df_step2["Name"] != order) & (df_step2["Numero Pagamento"] == n)]
    df_step2 = df_step2.drop(rows_to_drop.index, axis = 0)


df_step2 = df_step2.drop_duplicates(subset=['Name', 'Numero Pagamento'])
df_step2[["Name", "Paid at", "Total", "Payment Method", "Importo Pagato", "Numero Pagamento", "Data", "Time_difference"]]


,Name,Paid at,Total,Payment Method,Importo Pagato,Numero Pagamento,Data,Time_difference
2,#42263,2024-10-10 11:44:32 +0200,20.0,Qromo,20.0,d2a681ea-d9c2-484f-8546-a37483dc93b5,2024-10-10 11:44:23.805,0 days 00:00:08.195000
3,NaN,NaN,NaN,NaN,20.0,f4924aa1-79ec-4e55-980f-be0a32016c38,2024-10-18 15:52:29.093,NaT
4,#42526,2024-10-23 18:31:51 +0200,130.0,Qromo,130.0,e6082596-5ca6-4114-876f-937367fdf1f4,2024-10-23 18:31:32.479,0 days 00:00:18.521000
5,#42538,2024-10-24 14:44:45 +0200,45.0,Qromo,45.0,82862003-8ce7-42d5-8f31-8e0d3ec7c43e,2024-10-24 14:44:35.814,0 days 00:00:09.186000
0,#42165,2024-10-05 12:24:17 +0200,5.0,Qromo,NaN,NaN,NaT,NaT
1,#42244,2024-10-09 10:43:33 +0200,120.0,Qromo,NaN,NaN,NaT,NaT
6,#43006,2024-10-31 13:19:14 +0100,180.0,Qromo,NaN,NaN,NaT,NaT


In [319]:
set(df["Numero Pagamento"].unique()) - set(df_step2["Numero Pagamento"].unique())

set()

In [320]:
set(df_ordini_solo.Name.unique()) - set(df_step2.Name.unique())

set()

In [321]:
# Identify the names that are in df_ordini_solo but not in df_step1
missing_names = set(df_ordini_solo.Name.unique()) - set(df_step2.Name.unique())
missing_num = set(df["Numero Pagamento"].unique()) - set(df_step2["Numero Pagamento"].unique())

# Filter df_ordini_solo to include only the rows with missing names
missing_rows_step2 = df_ordini_solo[df_ordini_solo["Name"].isin(missing_names)]
missing_rows_num = df[df["Numero Pagamento"].isin(missing_num)]

In [322]:
df_steps = pd.concat([df_step2, missing_rows_step2, missing_rows_num])
df_steps

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,partial_date,Numero Pagamento,Data,Importo Pagato,description,Time_difference
2,#42263,manuela.facchinetti@fastwebnet.it,paid,2024-10-10 11:44:32 +0200,fulfilled,2024-10-10 11:44:33 +0200,no,EUR,20.0,0.0,3.61,20.0,NaN,0.0,NaN,2024-10-10 11:44:32 +0200,1.0,Repair Service LIL House - Saldatura bracciale...,20.0,NaN,1.579000e+10,False,True,fulfilled,Manuela Facchinetti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Qromo,rPktYjJvruE1naFEzyoV3xKir,0.0,LIL Milan,0.0,Veronica Varetta,LIL house,22.0,6.342991e+12,NaN,Low,pos,0.0,IT IVA 22%,3.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2633,NaN,NaN,NaN,rPktYjJvruE1naFEzyoV3xKir,NaN,NaN,rPktYjJvruE1naFEzyoV3xKir,2024-10-10,d2a681ea-d9c2-484f-8546-a37483dc93b5,2024-10-10 11:44:23.805,20.0,0.0,0 days 00:00:08.195000
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-18,f4924aa1-79ec-4e55-980f-be0a32016c38,2024-10-18 15:52:29.093,20.0,0.0,NaT
4,#42526,NaN,paid,2024-10-23 18:31:51 +0200,fulfilled,2024-10-30 19:10:03 +0100,no,EUR,250.0,0.0,45.08,130.0,NaN,0.0,NaN,2024-10-23 18:31:51 +0200,0.0,Pensavo fosse amore - Yellow / E,120.0,NaN,1.579000e+10,True,True,pending,Marco La Mantina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Qromo,rUglTyUvc8NHQdYS3yJgJF1ZH,0.0,LIL Milan,120.0,Veronica Varetta,LIL house,22.0,6.363635e+12,NaN,Low,pos,0.0,IT IVA 22%,45.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.936634e+11,22-2700,NaN,NaN,NaN,rUglTyUvc8NHQdYS3yJgJF1ZH,NaN,NaN,rUglTyUvc8NHQdYS3yJgJF1ZH,2024-10-23,e6082596-5ca6-4114-876f-937367fdf1f4,2024-10-23 18:31:32.479,130.0,0.0,0 days 00:00:18.521000
5,#42538,elena.assante@gmail.com,paid,2024-10-24 14:44:45 +0200,fulfilled,2024-10-24 14:44:45 +0200,yes,EUR,45.0,0.0,8.12,45.0,NaN,0.0,NaN,2024-10-24 14:44:44 +0200,1.0,Repair Service LIL House - Farfallina,25.0,NaN,1.579000e+10,True,True,fulfilled,elena assante,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Qromo,rRWcDgLy1nxW317tiK3DjkCRv,0.0,LIL Milan,0.0,Veronica Varetta,LIL house,22.0,6.365254e+12,NaN,Low,pos,0.0,IT IVA 22%,8.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2705,NaN,NaN,NaN,rRWcDgLy1nxW317tiK3DjkCRv,NaN,NaN,rRWcDgLy1nxW317tiK3DjkCRv,2024-10-24,82862003-8ce7-42d5-8f31-8e0d3ec7c43e,2024-10-24 14:44:35.814,45.0,0.0,0 days 00:00:09.186000
0,#42165,pietrogrigolo@gmail.com,paid,2024-10-05 12:24:17 +0200,fulfilled,2024-10-23 15:23:09 +0200,no,EUR,365.0,0.0,65.82,5.0,NaN,400.0,NaN,2024-10-05 12:24:17 +0200,0.0,Girls Tears Necklace - Yellow / 35cm,360.0,NaN,1.579000e+10,True,True,pending,Pietro Grigolo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

#### MATCH SOLO DATA

In [324]:
df_mix = df_steps[df_steps["Name"].isna()][["partial_date", "Data","Importo Pagato",	"Numero Pagamento"]]
df_steps = df_steps[~df_steps["Name"].isna()]
df_mix

,partial_date,Data,Importo Pagato,Numero Pagamento
3,2024-10-18,2024-10-18 15:52:29.093,20.0,f4924aa1-79ec-4e55-980f-be0a32016c38


In [326]:
df_check = pd.merge(df_ordini_mix, df_mix, on= "partial_date", how='outer', suffixes=('_qromo', '_ordini'))
df_check = df_check.drop_duplicates()

non_trovati = df_check[df_check["Name"].isna()][["partial_date", "Importo Pagato",	"Numero Pagamento"]]
df_check = df_check[~df_check["Name"].isna()]
# df_check = df_check[df_check["Total"] <= df_check["Importo Pagato"]]

df_check[["Name", "partial_date", "Paid at", "Total", "Payment Method", "Importo Pagato", "Numero Pagamento", "Data"]]

,Name,partial_date,Paid at,Total,Payment Method,Importo Pagato,Numero Pagamento,Data
0,#42439,2024-10-18,2024-10-18 15:52:35,120.0,Gift Card + Qromo,20.0,f4924aa1-79ec-4e55-980f-be0a32016c38,2024-10-18 15:52:29.093


In [331]:
# # Remove timezone from 'Paid at' and convert to datetime
df_check['Paid at'] = pd.to_datetime(df_check['Paid at']).dt.tz_localize(None) #.reset_index(drop=True).dt.tz_localize(None) 

# # Convert 'Data' column to datetime
df_check['Data'] = pd.to_datetime(df_check['Data'])#.reset_index(drop=True)

# # Calculate time difference, ignoring time zones
df_check['Time_difference'] = abs(df_check['Paid at']- df_check['Data'])

df_check = df_check.drop_duplicates(subset=["Name", "Numero Pagamento"])
df_check[["Name", "Paid at", "Total", "Payment Method", "Importo Pagato", "Numero Pagamento", "Data", "Time_difference"]]

,Name,Paid at,Total,Payment Method,Importo Pagato,Numero Pagamento,Data,Time_difference
0,#42439,2024-10-18 15:52:35,120.0,Gift Card + Qromo,20.0,f4924aa1-79ec-4e55-980f-be0a32016c38,2024-10-18 15:52:29.093,0 days 00:00:05.907000


In [332]:
df_step1 = df_check.sort_values("Paid at")
names = df_step1["Name"].unique()
    
for n in names:
    f = df_step1[(df_step1["Name"] == n)]
    # print(f['Time_difference'].dtype)

    if f['Time_difference'].isna().all():
        # print(f"All 'Time_difference' values are NaN for Numero Pagamento {n}")
        continue  # Skip this iteration if all values are NaN
    min_days_idx = f['Time_difference'].idxmin()
    order = f.loc[min_days_idx, "Numero Pagamento"]
    rows_to_drop = df_step1[(df_step1["Numero Pagamento"] == order) & (df_step1["Name"] != n) | (df_step1["Numero Pagamento"] != order) & (df_step1["Name"] == n)]
    df_step1 = df_step1.drop(rows_to_drop.index, axis = 0)


df_step1 = df_step1.drop_duplicates(subset=['Name', 'Numero Pagamento'])
df_step1[["Name", "Paid at", "Total", "Payment Method", "Importo Pagato", "Numero Pagamento", "Data", "Time_difference"]]

,Name,Paid at,Total,Payment Method,Importo Pagato,Numero Pagamento,Data,Time_difference
0,#42439,2024-10-18 15:52:35,120.0,Gift Card + Qromo,20.0,f4924aa1-79ec-4e55-980f-be0a32016c38,2024-10-18 15:52:29.093,0 days 00:00:05.907000


In [333]:
set(df_ordini_mix.Name.unique()) - set(df_step1.Name.unique())

set()

In [334]:
set(df_mix["Numero Pagamento"].unique()) - set(df_step1["Numero Pagamento"].unique())

set()

In [335]:
# Identify the names that are in df_ordini_solo but not in df_step1
missing_names = set(df_ordini_mix.Name.unique()) - set(df_step1.Name.unique())
missing_num = set(df_mix["Numero Pagamento"].unique()) - set(df_step1["Numero Pagamento"].unique())

# Filter df_ordini_solo to include only the rows with missing names
missing_rows_step1 = df_ordini_mix[df_ordini_mix["Name"].isin(missing_names)]
missing_rows_num = df_mix[df_mix["Numero Pagamento"].isin(missing_num)]

In [336]:
df_final = pd.concat([df_steps, df_step1, missing_rows_step1, missing_rows_num])

In [337]:
df_final = df_final.drop_duplicates(subset=["Name", "Numero Pagamento"])

In [338]:
df_final.loc[df_final["Total"] == df_final["Importo Pagato"], "CHECK"] = "VERO"
df_final.loc[df_final["Total"] != df_final["Importo Pagato"], "CHECK"] = "FALSO"
df_final.loc[df_final["Name"].isna() | df_final["Numero Pagamento"].isna(), "CHECK"] = "NON TROVATO"

df_final["CHECK"].value_counts()

CHECK
VERO           3
NON TROVATO    3
FALSO          1
Name: count, dtype: int64

In [339]:
df_final

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,partial_date,Numero Pagamento,Data,Importo Pagato,description,Time_difference,CHECK
2,#42263,manuela.facchinetti@fastwebnet.it,paid,2024-10-10 11:44:32 +0200,fulfilled,2024-10-10 11:44:33 +0200,no,EUR,20.0,0.0,3.61,20.0,NaN,0.0,NaN,2024-10-10 11:44:32 +0200,1.0,Repair Service LIL House - Saldatura bracciale...,20.0,NaN,1.579000e+10,False,True,fulfilled,Manuela Facchinetti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Qromo,rPktYjJvruE1naFEzyoV3xKir,0.0,LIL Milan,0.0,Veronica Varetta,LIL house,22.0,6.342991e+12,NaN,Low,pos,0.0,IT IVA 22%,3.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2633,NaN,NaN,NaN,rPktYjJvruE1naFEzyoV3xKir,NaN,NaN,rPktYjJvruE1naFEzyoV3xKir,2024-10-10,d2a681ea-d9c2-484f-8546-a37483dc93b5,2024-10-10 11:44:23.805,20.0,0.0,0 days 00:00:08.195000,VERO
4,#42526,NaN,paid,2024-10-23 18:31:51 +0200,fulfilled,2024-10-30 19:10:03 +0100,no,EUR,250.0,0.0,45.08,130.0,NaN,0.0,NaN,2024-10-23 18:31:51 +0200,0.0,Pensavo fosse amore - Yellow / E,120.0,NaN,1.579000e+10,True,True,pending,Marco La Mantina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Qromo,rUglTyUvc8NHQdYS3yJgJF1ZH,0.0,LIL Milan,120.0,Veronica Varetta,LIL house,22.0,6.363635e+12,NaN,Low,pos,0.0,IT IVA 22%,45.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.936634e+11,22-2700,NaN,NaN,NaN,rUglTyUvc8NHQdYS3yJgJF1ZH,NaN,NaN,rUglTyUvc8NHQdYS3yJgJF1ZH,2024-10-23,e6082596-5ca6-4114-876f-937367fdf1f4,2024-10-23 18:31:32.479,130.0,0.0,0 days 00:00:18.521000,VERO
5,#42538,elena.assante@gmail.com,paid,2024-10-24 14:44:45 +0200,fulfilled,2024-10-24 14:44:45 +0200,yes,EUR,45.0,0.0,8.12,45.0,NaN,0.0,NaN,2024-10-24 14:44:44 +0200,1.0,Repair Service LIL House - Farfallina,25.0,NaN,1.579000e+10,True,True,fulfilled,elena assante,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Qromo,rRWcDgLy1nxW317tiK3DjkCRv,0.0,LIL Milan,0.0,Veronica Varetta,LIL house,22.0,6.365254e+12,NaN,Low,pos,0.0,IT IVA 22%,8.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2705,NaN,NaN,NaN,rRWcDgLy1nxW317tiK3DjkCRv,NaN,NaN,rRWcDgLy1nxW317tiK3DjkCRv,2024-10-24,82862003-8ce7-42d5-8f31-8e0d3ec7c43e,2024-10-24 14:44:35.814,45.0,0.0,0 days 00:00:09.186000,VERO
0,#42165,pietrogrigolo@gmail.com,paid,2024-10-05 12:24:17 +0200,fulfilled,2024-10-23 15:23:09 +0200,no,EUR,365.0,0.0,65.82,5.0,NaN,400.0,NaN,2024-10-05 12:24:17 +0200,0.0,Girls Tears Necklace - Yellow / 35cm,360.0,NaN,1.579000e+10,True,True,pending,Pietro Grigolo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Qromo,reUdxgh25VLSxfKF1p5gGUp5H,0.0,LIL Milan,0.0,Veronica Varetta,LIL house,22.0,6.335479e+12,NaN,Low,pos,0.0,IT IVA 22%,65.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2617,NaN,NaN,NaN,reUdxgh25VLSxfKF1p5gGUp5H,NaN,NaN,reUdxgh25VLSxfKF1p5gGUp5H,2024-10-05,NaN,NaT,NaN,NaN,NaT,NON TROVATO
1,#42244,NaN,paid,2024-10-09 10:43:33 +0200,fulfilled,202

### BONIFICO

In [359]:
b = pd.read_excel(filepath+"Intesa 10.2024.xlsx")

In [360]:
operations_patterns = [
            r'stripe',
            r'paypal',
            r'satispay',
            r'scalapay',
            r'rinascente',
            r'paesi ue',
            r'retail group',
            r'sportello automatico',
        ]

mask = ~b['Operazione'].str.contains('|'.join(operations_patterns), case=False, regex=True, na=False)
b = b[mask]

df = b.copy()
df = df.rename(columns={"Dettagli": "Numero Pagamento", 'Importo': 'Importo Pagato'})

In [362]:
df['partial_date'] =  pd.to_datetime(df['Data']).dt.tz_localize(None).dt.date
df = df.sort_values('Data')

o["Payment Method"] = o["Payment Method"].str.strip()
o["Payment Method"] = o["Payment Method"].str.replace("custom|Wire Transfer", "Bonifico", regex=True)

df_ordini = o[o['Payment Method'].str.contains('Bonifico', case=False, na=False)]
paid_at = df_ordini['Paid at'].str.replace(r'\s[+-]\d{4}$', '', regex=True)
df_ordini['partial_date'] = pd.to_datetime(paid_at).dt.tz_localize(None).dt.date
df_ordini = df_ordini.sort_values('Paid at')

In [363]:
df_ordini_solo = df_ordini[df_ordini["Payment Method"] == "Bonifico"]
df_ordini_mix = df_ordini[(df_ordini["Payment Method"] != "Bonifico")]
print(len(df_ordini_mix.Name.unique()), len(df_ordini_solo.Name.unique()))

0 17


#### MATCH SOLO DATA ORDINI SOLO BONIFICO

In [374]:
df_check = pd.merge(df_ordini_solo, df, left_on='Total', right_on='Importo Pagato', how='outer')
df_check = df_check.drop_duplicates()

df_check[["Name", "Paid at", "Total", "Payment Method", "Importo Pagato", "Numero Pagamento", "Data"]]

,Name,Paid at,Total,Payment Method,Importo Pagato,Numero Pagamento,Data
0,#42098,2024-10-02 06:38:32 +0200,115.00,Bonifico,115.0,COD. DISP. 0124100168917586 CASH 02INTER202410...,2024-10-01
1,#42098,2024-10-02 06:38:32 +0200,115.00,Bonifico,115.0,COD. DISP. 0124100168917586 CASH 02INTER202410...,2024-10-01
2,#42932,NaN,122.00,Bonifico,NaN,NaN,NaT
3,#42321,2024-10-11 17:19:41 +0200,150.00,Bonifico,150.0,COD. DISP. 0124101173330000 CASH 02INTER202410...,2024-10-11
4,#42483,NaN,150.00,Bonifico,150.0,COD. DISP. 0124101173330000 CASH 02INTER202410...,2024-10-11
5,#42483,NaN,150.00,Bonifico,150.0,COD. DISP. 0124101173330000 CASH 02INTER202410...,2024-10-11
6,NaN,NaN,NaN,NaN,167.0,COD. DISP. 0124101846638754 CASH NOTPROVIDED O...,2024-10-18
7,#42945,2024-10-29 07:42:54 +0100,208.00,Bonifico,208.0,COD. DISP. 0124102821951899 CASH NOTPROVIDED O...,2024-10-28
8,#42945,2024-10-29 07:42:54 +0100,208.00,Bonifico,208.0,COD. DISP. 0124103058302651 CASH 0000028768326...,2024-10-30
9,#42765,2024-10-31 18:27:31 +0100,208.00,Bonifico,208.0,COD. DISP. 0124102821951899 CASH NOTPROVIDED O...,2024-10-28


In [375]:
# # Remove timezone from 'Paid at' and convert to datetime
df_check['Paid at'] = pd.to_datetime(df_check['Paid at'].str.replace(r'\s[+-]\d{4}$', '', regex=True)).reset_index(drop=True)#.reset_index(drop=True).dt.tz_localize(None) 

# # Convert 'Data' column to datetime
df_check['Data'] = pd.to_datetime(df_check['Data'])#.reset_index(drop=True)

# # Calculate time difference, ignoring time zones
df_check['Time_difference'] = df_check['Paid at']- df_check['Data']
df_check = df_check[(df_check['Time_difference'] >= pd.Timedelta(0)) | df_check['Time_difference'].isna()]

df_check = df_check.drop_duplicates(subset=["Name", "Numero Pagamento"])
df_check[["Name", "Paid at", "Total", "Payment Method", "Importo Pagato", "Numero Pagamento", "Data", "Time_difference"]]

,Name,Paid at,Total,Payment Method,Importo Pagato,Numero Pagamento,Data,Time_difference
0,#42098,2024-10-02 06:38:32,115.00,Bonifico,115.0,COD. DISP. 0124100168917586 CASH 02INTER202410...,2024-10-01,1 days 06:38:32
2,#42932,NaT,122.00,Bonifico,NaN,NaN,NaT,NaT
3,#42321,2024-10-11 17:19:41,150.00,Bonifico,150.0,COD. DISP. 0124101173330000 CASH 02INTER202410...,2024-10-11,0 days 17:19:41
4,#42483,NaT,150.00,Bonifico,150.0,COD. DISP. 0124101173330000 CASH 02INTER202410...,2024-10-11,NaT
6,NaN,NaT,NaN,NaN,167.0,COD. DISP. 0124101846638754 CASH NOTPROVIDED O...,2024-10-18,NaT
7,#42945,2024-10-29 07:42:54,208.00,Bonifico,208.0,COD. DISP. 0124102821951899 CASH NOTPROVIDED O...,2024-10-28,1 days 07:42:54
9,#42765,2024-10-31 18:27:31,208.00,Bonifico,208.0,COD. DISP. 0124102821951899 CASH NOTPROVIDED O...,2024-10-28,3 days 18:27:31
10,#42765,2024-10-31 18:27:31,208.00,Bonifico,208.0,COD. DISP. 0124103058302651 CASH 0000028768326...,2024-10-30,1 days 18:27:31
11,#42379,2024-10-15 15:32:09,240.00,Bonifico,NaN,NaN,NaT,NaT
12,#42226,2024-10-09 10:00:13,252.00,Bonifico,252.0,COD. DISP. 0124100871778432 CASH 02INTER202410...,2024-10-08,1 days 10:00:13


In [376]:
df_step1 = df_check.sort_values("Paid at")
names = df_step1["Name"].unique()
    
for n in names:
    f = df_step1[(df_step1["Name"] == n)]
    # print(f['Time_difference'].dtype)

    if f['Time_difference'].isna().all():
        # print(f"All 'Time_difference' values are NaN for Numero Pagamento {n}")
        continue  # Skip this iteration if all values are NaN
    min_days_idx = f['Time_difference'].idxmin()
    order = f.loc[min_days_idx, "Numero Pagamento"]
    rows_to_drop = df_step1[(df_step1["Numero Pagamento"] == order) & (df_step1["Name"] != n) | (df_step1["Numero Pagamento"] != order) & (df_step1["Name"] == n)]
    df_step1 = df_step1.drop(rows_to_drop.index, axis = 0)


df_step1 = df_step1.drop_duplicates(subset=['Name', 'Numero Pagamento'])
df_step1[["Name", "Paid at", "Total", "Payment Method", "Importo Pagato", "Numero Pagamento", "Data", "Time_difference"]]

,Name,Paid at,Total,Payment Method,Importo Pagato,Numero Pagamento,Data,Time_difference
0,#42098,2024-10-02 06:38:32,115.00,Bonifico,115.0,COD. DISP. 0124100168917586 CASH 02INTER202410...,2024-10-01,1 days 06:38:32
12,#42226,2024-10-09 10:00:13,252.00,Bonifico,252.0,COD. DISP. 0124100871778432 CASH 02INTER202410...,2024-10-08,1 days 10:00:13
3,#42321,2024-10-11 17:19:41,150.00,Bonifico,150.0,COD. DISP. 0124101173330000 CASH 02INTER202410...,2024-10-11,0 days 17:19:41
22,#42340,2024-10-13 11:30:17,330.00,Bonifico,330.0,COD. DISP. 0124101373598034 CASH NOTPROVIDED O...,2024-10-12,1 days 11:30:17
11,#42379,2024-10-15 15:32:09,240.00,Bonifico,NaN,NaN,NaT,NaT
13,#42436,2024-10-23 09:19:31,290.00,Bonifico,290.0,COD. DISP. 0124102254947713 CASH NOTPROVIDED O...,2024-10-22,1 days 09:19:31
18,#42555,2024-10-25 16:44:14,320.00,Bonifico,320.0,COD. DISP. 0124102459648918 CASH Consuelo Rega...,2024-10-24,1 days 16:44:14
7,#42945,2024-10-29 07:42:54,208.00,Bonifico,208.0,COD. DISP. 0124102821951899 CASH NOTPROVIDED O...,2024-10-28,1 days 07:42:54
30,#42640,2024-10-29 13:54:59,500.00,Bonifico,500.0,COD. DISP. 0124102930189541 CASH LILIANA LIETT...,2024-10-29,0 days 13:54:59
10,#42765,2024-10-31 18:27:31,208.00,Bonifico,208.0,COD. DISP. 0124103058302651 CASH 0000028768326...,2024-10-30,1 days 18:27:31


In [ ]:
set(df_ordini_mix.Name.unique()) - set(df_step1.Name.unique())

set()

In [ ]:
set(df_mix["Numero Pagamento"].unique()) - set(df_step1["Numero Pagamento"].unique())

set()

In [ ]:
# Identify the names that are in df_ordini_solo but not in df_step1
missing_names = set(df_ordini_mix.Name.unique()) - set(df_step1.Name.unique())
missing_num = set(df_mix["Numero Pagamento"].unique()) - set(df_step1["Numero Pagamento"].unique())

# Filter df_ordini_solo to include only the rows with missing names
missing_rows_step1 = df_ordini_mix[df_ordini_mix["Name"].isin(missing_names)]
missing_rows_num = df_mix[df_mix["Numero Pagamento"].isin(missing_num)]

In [ ]:
df_final = pd.concat([df_steps, df_step1, missing_rows_step1, missing_rows_num])

In [ ]:
df_final = df_final.drop_duplicates(subset=["Name", "Numero Pagamento"])

In [ ]:
df_final.loc[df_final["Total"] == df_final["Importo Pagato"], "CHECK"] = "VERO"
df_final.loc[df_final["Total"] != df_final["Importo Pagato"], "CHECK"] = "FALSO"
df_final.loc[df_final["Name"].isna() | df_final["Numero Pagamento"].isna(), "CHECK"] = "NON TROVATO"

df_final["CHECK"].value_counts()

CHECK
VERO           3
NON TROVATO    3
FALSO          1
Name: count, dtype: int64

In [ ]:
df_final

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,partial_date,Numero Pagamento,Data,Importo Pagato,description,Time_difference,CHECK
2,#42263,manuela.facchinetti@fastwebnet.it,paid,2024-10-10 11:44:32 +0200,fulfilled,2024-10-10 11:44:33 +0200,no,EUR,20.0,0.0,3.61,20.0,NaN,0.0,NaN,2024-10-10 11:44:32 +0200,1.0,Repair Service LIL House - Saldatura bracciale...,20.0,NaN,1.579000e+10,False,True,fulfilled,Manuela Facchinetti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Qromo,rPktYjJvruE1naFEzyoV3xKir,0.0,LIL Milan,0.0,Veronica Varetta,LIL house,22.0,6.342991e+12,NaN,Low,pos,0.0,IT IVA 22%,3.61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2633,NaN,NaN,NaN,rPktYjJvruE1naFEzyoV3xKir,NaN,NaN,rPktYjJvruE1naFEzyoV3xKir,2024-10-10,d2a681ea-d9c2-484f-8546-a37483dc93b5,2024-10-10 11:44:23.805,20.0,0.0,0 days 00:00:08.195000,VERO
4,#42526,NaN,paid,2024-10-23 18:31:51 +0200,fulfilled,2024-10-30 19:10:03 +0100,no,EUR,250.0,0.0,45.08,130.0,NaN,0.0,NaN,2024-10-23 18:31:51 +0200,0.0,Pensavo fosse amore - Yellow / E,120.0,NaN,1.579000e+10,True,True,pending,Marco La Mantina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Qromo,rUglTyUvc8NHQdYS3yJgJF1ZH,0.0,LIL Milan,120.0,Veronica Varetta,LIL house,22.0,6.363635e+12,NaN,Low,pos,0.0,IT IVA 22%,45.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.936634e+11,22-2700,NaN,NaN,NaN,rUglTyUvc8NHQdYS3yJgJF1ZH,NaN,NaN,rUglTyUvc8NHQdYS3yJgJF1ZH,2024-10-23,e6082596-5ca6-4114-876f-937367fdf1f4,2024-10-23 18:31:32.479,130.0,0.0,0 days 00:00:18.521000,VERO
5,#42538,elena.assante@gmail.com,paid,2024-10-24 14:44:45 +0200,fulfilled,2024-10-24 14:44:45 +0200,yes,EUR,45.0,0.0,8.12,45.0,NaN,0.0,NaN,2024-10-24 14:44:44 +0200,1.0,Repair Service LIL House - Farfallina,25.0,NaN,1.579000e+10,True,True,fulfilled,elena assante,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Qromo,rRWcDgLy1nxW317tiK3DjkCRv,0.0,LIL Milan,0.0,Veronica Varetta,LIL house,22.0,6.365254e+12,NaN,Low,pos,0.0,IT IVA 22%,8.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2705,NaN,NaN,NaN,rRWcDgLy1nxW317tiK3DjkCRv,NaN,NaN,rRWcDgLy1nxW317tiK3DjkCRv,2024-10-24,82862003-8ce7-42d5-8f31-8e0d3ec7c43e,2024-10-24 14:44:35.814,45.0,0.0,0 days 00:00:09.186000,VERO
0,#42165,pietrogrigolo@gmail.com,paid,2024-10-05 12:24:17 +0200,fulfilled,2024-10-23 15:23:09 +0200,no,EUR,365.0,0.0,65.82,5.0,NaN,400.0,NaN,2024-10-05 12:24:17 +0200,0.0,Girls Tears Necklace - Yellow / 35cm,360.0,NaN,1.579000e+10,True,True,pending,Pietro Grigolo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Qromo,reUdxgh25VLSxfKF1p5gGUp5H,0.0,LIL Milan,0.0,Veronica Varetta,LIL house,22.0,6.335479e+12,NaN,Low,pos,0.0,IT IVA 22%,65.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2617,NaN,NaN,NaN,reUdxgh25VLSxfKF1p5gGUp5H,NaN,NaN,reUdxgh25VLSxfKF1p5gGUp5H,2024-10-05,NaN,NaT,NaN,NaN,NaT,NON TROVATO
1,#42244,NaN,paid,2024-10-09 10:43:33 +0200,fulfilled,202

In [36]:
def choose_merges2(rest):
    names = rest["Name"].unique()
    
    for n in names:
        f = rest[(rest["Name"] == n)]
        # print(f['Time_difference'].dtype)

        if f['Time_difference'].isna().all():
            print(f"All 'Time_difference' values are NaN for Numero Pagamento {n}")
            continue  # Skip this iteration if all values are NaN
        # min_value = f['Time_difference_seconds'].min()
        # min_row = f[f['Time_difference_seconds'] == min_value]
        min_days_idx = f['Time_difference'].idxmin()
        if f.loc[min_days_idx, "Importo Pagato"] != f.loc[min_days_idx, "Total"]:
            continue
        order = f.loc[min_days_idx, "Numero Pagamento"]
        # order = min_row["Name"]
        rows_to_drop = rest[(df_check["Numero Pagamento"] == order) & (rest["Name"] != n) | (rest["Numero Pagamento"] != order) & (rest["Name"] == n)]
        rest = rest.drop(rows_to_drop.index, axis = 0)

    return rest

In [32]:
df = pd.concat([prova, rest])

names = set(df.Name.unique())
names2 = set(o.Name.unique())


names = set(df["Numero Pagamento"].unique())
names2 = set(q.ID.unique())

names2-names

{'pi_3Q52rfL1BrR61arW0YFGK03a',
 'pi_3Q7e3mL1BrR61arW0T8COwc0',
 'pi_3Q7wFhL1BrR61arW1T5CqSdu',
 'pi_3Q806lL1BrR61arW1x3OjV6Z',
 'pi_3Q96g8L1BrR61arW16E4TqPP',
 'pi_3QAW47L1BrR61arW18UQ5Kdo',
 'pi_3QClIRL1BrR61arW0NZm05jX',
 'pi_3QED2VL1BrR61arW1crlrSQV',
 'pi_3QFdLWL1BrR61arW1Z5jIUcI',
 'pi_3QFdLvL1BrR61arW0KvFDID3'}

In [41]:
prova = choose_merges2(df_check)
prova[["Name", "Paid at", "Total", "Payment Method", "Importo Pagato", "Numero Pagamento", "Data", "Time_difference"]]

All 'Time_difference' values are NaN for Numero Pagamento #42344
All 'Time_difference' values are NaN for Numero Pagamento #42398
All 'Time_difference' values are NaN for Numero Pagamento #42439
All 'Time_difference' values are NaN for Numero Pagamento #42538


,Name,Paid at,Total,Payment Method,Importo Pagato,Numero Pagamento,Data,Time_difference
1,#22-2600,2024-10-01 17:08:48 +0200,1400.0,Qromo,1400.0,pi_3Q57gVL1BrR61arW17mWj2gF,2024-10-01 17:08:19,0 days 00:00:29
6,#22-2601,2024-10-01 18:36:14 +0200,320.0,Qromo,320.0,pi_3Q592wL1BrR61arW0q9AAp01,2024-10-01 18:35:34,0 days 00:00:40
11,#42113,2024-10-01 18:47:59 +0200,120.0,Qromo,120.0,pi_3Q59EdL1BrR61arW0wsFm270,2024-10-01 18:47:39,0 days 00:00:20
12,#42128,2024-10-02 13:12:01 +0200,20.0,Qromo,20.0,pi_3Q5QT2L1BrR61arW1cvyeAL2,2024-10-02 13:11:40,0 days 00:00:21
16,#42131,2024-10-02 13:33:54 +0200,125.0,Qromo,125.0,pi_3Q5Qo8L1BrR61arW1r57vF31,2024-10-02 13:33:28,0 days 00:00:26
...,...,...,...,...,...,...,...,...
1009,#43006,2024-10-31 13:19:14 +0100,420.0,Qromo,240.0,pi_3QFxKyL1BrR61arW0HviTjrm,2024-10-31 13:18:52,0 days 00:00:22
1015,#43006,2024-10-31 13:19:14 +0100,420.0,Qromo,240.0,pi_3QFxKyL1BrR61arW0HviTjrm,2024-10-31 13:18:52,0 days 00:00:22
1022,#22-2754,2024-10-31 13:38:53 +0100,240.0,Qromo,240.0,pi_3QFxdpL1BrR61arW0nOFe5d8,2024-10-31 13:38:21,0 days 00:00:32
1029,#43008,2024-10-31 13:42:03 +0100,280.0,Qromo,280.0,pi_3QFxh3L1BrR61arW1xn614XT,2024-10-31 13:41:41,0 days 00:00:22


In [44]:
prova = prova.drop_duplicates(subset=["Numero Pagamento", "Name"])

rest = prova[prova["Total"] != prova["Importo Pagato"]]
rest[["Name", "Paid at", "Total", "Payment Method", "Importo Pagato", "Numero Pagamento", "Data", "Time_difference"]]

,Name,Paid at,Total,Payment Method,Importo Pagato,Numero Pagamento,Data,Time_difference
142,#42174,2024-10-05 16:53:49 +0200,140.0,Gift Card + Qromo,15.0,pi_3Q6ZMPL1BrR61arW1HNGWtQx,2024-10-05 16:53:33,0 days 00:00:16
197,#42233,2024-10-08 14:47:07 +0200,1100.0,Cash + Qromo,400.0,pi_3Q7cnyL1BrR61arW0dp4bGtH,2024-10-08 14:46:22,0 days 00:00:45
347,#42311,2024-10-11 18:35:26 +0200,240.0,Qromo + manual,100.0,pi_3Q8gztL1BrR61arW1tL4lmCi,2024-10-11 13:27:05,0 days 05:08:21
350,#42311,2024-10-11 18:35:26 +0200,240.0,Qromo + manual,40.0,pi_3Q8lcHL1BrR61arW1TRt6nf3,2024-10-11 18:23:01,0 days 00:12:25
578,#42520,2024-10-23 17:32:01 +0200,640.0,Qromo,320.0,pi_3QD6WjL1BrR61arW1cQ1CDei,2024-10-23 17:31:13,0 days 00:00:48
590,#42526,2024-10-23 18:31:51 +0200,250.0,Qromo,320.0,pi_3QD6WjL1BrR61arW1cQ1CDei,2024-10-23 17:31:13,0 days 01:00:38
643,#42497,2024-10-24 17:45:21 +0200,460.0,Qromo + manual,20.0,pi_3QDTBgL1BrR61arW1I4fvVUD,2024-10-24 17:43:00,0 days 00:02:21
1009,#43006,2024-10-31 13:19:14 +0100,420.0,Qromo,240.0,pi_3QFxKyL1BrR61arW0HviTjrm,2024-10-31 13:18:52,0 days 00:00:22


In [46]:
    
names = rest["Numero Pagamento"].unique()

for n in names:
    f = rest[(rest["Numero Pagamento"] == n)]
    # print(f['Time_difference'].dtype)

    if f['Time_difference'].isna().all():
        print(f"All 'Time_difference' values are NaN for Numero Pagamento {n}")
        continue  # Skip this iteration if all values are NaN
    # min_value = f['Time_difference_seconds'].min()
    # min_row = f[f['Time_difference_seconds'] == min_value]
    min_days_idx = f['Time_difference'].idxmin()
    order = f.loc[min_days_idx, "Name"]
    # order = min_row["Name"]rest
    rows_to_drop = rest[(rest["Name"] == order) & (rest["Numero Pagamento"] != n) | (rest["Name"] != order) & (rest["Numero Pagamento"] == n)]
    rest = rest.drop(rows_to_drop.index, axis = 0)
    

All 'Time_difference' values are NaN for Numero Pagamento pi_3Q8lcHL1BrR61arW1TRt6nf3


In [47]:

rest[["Name", "Paid at", "Total", "Payment Method", "Importo Pagato", "Numero Pagamento", "Data", "Time_difference"]]

,Name,Paid at,Total,Payment Method,Importo Pagato,Numero Pagamento,Data,Time_difference
142,#42174,2024-10-05 16:53:49 +0200,140.0,Gift Card + Qromo,15.0,pi_3Q6ZMPL1BrR61arW1HNGWtQx,2024-10-05 16:53:33,0 days 00:00:16
197,#42233,2024-10-08 14:47:07 +0200,1100.0,Cash + Qromo,400.0,pi_3Q7cnyL1BrR61arW0dp4bGtH,2024-10-08 14:46:22,0 days 00:00:45
347,#42311,2024-10-11 18:35:26 +0200,240.0,Qromo + manual,100.0,pi_3Q8gztL1BrR61arW1tL4lmCi,2024-10-11 13:27:05,0 days 05:08:21
578,#42520,2024-10-23 17:32:01 +0200,640.0,Qromo,320.0,pi_3QD6WjL1BrR61arW1cQ1CDei,2024-10-23 17:31:13,0 days 00:00:48
643,#42497,2024-10-24 17:45:21 +0200,460.0,Qromo + manual,20.0,pi_3QDTBgL1BrR61arW1I4fvVUD,2024-10-24 17:43:00,0 days 00:02:21
1009,#43006,2024-10-31 13:19:14 +0100,420.0,Qromo,240.0,pi_3QFxKyL1BrR61arW0HviTjrm,2024-10-31 13:18:52,0 days 00:00:22


In [29]:
q[q["Data"].str.startswith("2024-10-09")]

,Data,Stato,Importo €,Importo rimborsato €,ID,Importo Pagato,Numero Pagamento
108,2024-10-09 15:39:19,Riuscito,794.0,0.0,pi_3Q806lL1BrR61arW1x3OjV6Z,794.0,pi_3Q806lL1BrR61arW1x3OjV6Z
109,2024-10-09 11:32:17,Riuscito,40.0,0.0,pi_3Q7wFhL1BrR61arW1T5CqSdu,40.0,pi_3Q7wFhL1BrR61arW1T5CqSdu
110,2024-10-09 11:03:42,Riuscito,10.0,0.0,pi_3Q7vo2L1BrR61arW0udLpGQa,10.0,pi_3Q7vo2L1BrR61arW0udLpGQa


In [48]:
df2 = pd.concat([prova, rest])

In [52]:
df.equals(df2)

True

In [53]:
df.Time_difference.describe()

count                          119
mean     0 days 00:06:23.739495798
std      0 days 00:40:03.090547050
min                0 days 00:00:14
25%                0 days 00:00:21
50%                0 days 00:00:27
75%                0 days 00:00:37
max                0 days 05:08:21
Name: Time_difference, dtype: object

In [27]:
o[o.Name == "#42244"]

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References
1177,#42244,NaN,paid,2024-10-09 10:43:33 +0200,fulfilled,2024-10-09 10:43:34 +0200,no,EUR,120.0,0.0,21.64,120.0,NaN,0.0,NaN,2024-10-09 10:43:32 +0200,1,Say My Name Piercing - Yellow / X / None,120.0,150.0,1.579000e+10,True,True,fulfilled,carlotta trentin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Qromo,r9aaZrala8SkQCqtQaS0Qjmkd,0.0,LIL Milan,0.0,Carlotta Trentin,Firgun House,24.0,6.341405e+12,NaN,Low,pos,0.0,IT IVA 22%,21.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24-1069,NaN,NaN,NaN,r9aaZrala8SkQCqtQaS0Qjmkd,NaN,NaN,r9aaZrala8SkQCqtQaS0Qjmkd


In [28]:
r = o[o["Location"].isin(["LIL Rinascente Milano", "LIL Rinascente Torino"])]

reso = r[r["Lineitem compare at price"] == 0]["Name"]
r.loc[(r["Name"] == reso.iloc[0]) & (r["Lineitem compare at price"] != 0), "Lineitem quantity"] = 0


In [29]:
r

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,Giorno
4,#43015,NaN,paid,NaN,fulfilled,2024-10-31 18:48:30 +0100,no,EUR,0.0,0.0,0.0,0.0,TORINO100%,100.0,NaN,2024-10-31 18:48:29 +0100,1,Nude Ring - Yellow / 10,100.0,NaN,1.579000e+10,True,True,fulfilled,Irene Moscato,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cv,NaN,NaN,NaN,NaN,0.0,LIL Milan,0.0,Veronica Varetta,LIL Rinascente Torino,3.0,6.377702e+12,NaN,Low,pos,0.0,IT IVA 0%,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3-5774,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,#43011,luca.bongiovanni90@gmail.com,paid,NaN,fulfilled,2024-10-31 17:02:54 +0100,no,EUR,0.0,0.0,0.0,0.0,TORINO100%,320.0,NaN,2024-10-31 17:02:53 +0100,1,Boys Tears Necklace - Yellow / 35cm,320.0,NaN,1.579000e+10,True,True,fulfilled,Luca Bongiovanni,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cv,NaN,NaN,NaN,NaN,0.0,LIL Milan,0.0,Veronica Varetta,LIL Rinascente Torino,3.0,6.377554e+12,NaN,Low,pos,0.0,IT IVA 0%,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3-5773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,#43009,NaN,paid,NaN,fulfilled,2024-10-31 15:53:46 +0100,no,EUR,0.0,0.0,0.0,0.0,TORINO100%,100.0,NaN,2024-10-31 15:53:45 +0100,1,Lightly Ring - White / 13,100.0,NaN,1.579000e+10,True,True,fulfilled,Ylenia Gugliotta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cv,NaN,NaN,NaN,NaN,0.0,LIL Milan,0.0,Veronica Varetta,LIL Rinascente Torino,3.0,6.377457e+12,NaN,Low,pos,0.0,IT IVA 0%,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3-5772,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,#42998,sara.masoero96@gmail.com,paid,NaN,fulfilled,2024-10-30 20:01:26 +0100,no,EUR,0.0,0.0,0.0,0.0,TORINO100%,260.0,NaN,2024-10-30 20:01:25 +0100,1,Balmy Necklace - Yellow / 36cm,260.0,NaN,1.579000e+10,True,True,fulfilled,Sara Masoero,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cce,NaN,NaN,NaN,NaN,0.0,LIL Milan,0.0,Veronica Varetta,LIL Rinascente Torino,3.0,6.375859e+12,NaN,Low,pos,0.0,IT IVA 0%,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3-5771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
432,#42721,saraperrino@hotmail.it,paid,NaN,fulfilled,2024-10-26 19:09:37 +0200,yes,EUR,0.0,0.0,0.0,0.0,TORINO100%,280.0,NaN,2024-10-26 19:09:37 +0200,1,Calypso Ring - Yellow / 10,280.0,NaN,1.579000e+10,True,True,fulfilled,Sara nicole Perrino,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cv,NaN,NaN,NaN,NaN,0.0,LIL Milan,0.0,Veronica Varetta,LIL Rinascente Torino,3.0,6.368384e+12,NaN,Low,pos,0.0,IT IVA 22%,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3-5769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397,#42108,NaN,paid,NaN,fulfilled,2024-10-01 

In [36]:
r[r["Giorno"] == "2024-10-07"]

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,Giorno,Totale
1213,#42220,luisa.lucato@gmail.com,paid,NaN,fulfilled,2024-10-07 19:08:40 +0200,no,EUR,0.0,0.0,0.0,0.0,MILANO100%,320.0,NaN,2024-10-07 19:08:39 +0200,1,Sunshine Ring - Yellow / 10 / White,320.0,NaN,1.579000e+10,True,True,fulfilled,Luisa Lucato,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,LIL Milan,0.0,Veronica Varetta,LIL Rinascente Milano,23.0,6.339267e+12,acquisto-rinascente-milano,Low,pos,0.0,IT IVA 22%,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23-2522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-07,320.0
1226,#42213,NaN,paid,NaN,fulfilled,2024-10-07 12:09:18 +0200,no,EUR,0.0,0.0,0.0,0.0,MILANO100%,980.0,NaN,2024-10-07 12:09:17 +0200,1,Honey Choker - Yellow,980.0,NaN,1.579000e+10,True,True,fulfilled,Mariana Hoyos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,NaN,NaN,NaN,NaN,0.0,LIL Milan,0.0,Veronica Varetta,LIL Rinascente Milano,23.0,6.338484e+12,acquisto-rinascente-milano,Low,pos,0.0,IT IVA 22%,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23-2521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-07,980.0


In [35]:
r["Giorno"] = r["Fulfilled at"].str[:10]
r["Totale"] = r["Lineitem quantity"] * r["Lineitem price"]

dates = r.groupby(["Giorno", "Location"])["Totale"].sum()
dates

Giorno      Location             
2024-10-01  LIL Rinascente Milano     420.0
            LIL Rinascente Torino    1340.0
2024-10-02  LIL Rinascente Milano     815.0
            LIL Rinascente Torino     165.0
2024-10-03  LIL Rinascente Torino     400.0
2024-10-04  LIL Rinascente Torino     410.0
2024-10-05  LIL Rinascente Milano    2840.0
            LIL Rinascente Torino     160.0
2024-10-06  LIL Rinascente Milano    6305.0
            LIL Rinascente Torino     905.0
2024-10-07  LIL Rinascente Milano    1300.0
2024-10-08  LIL Rinascente Milano     100.0
            LIL Rinascente Torino     480.0
2024-10-09  LIL Rinascente Milano     660.0
2024-10-10  LIL Rinascente Milano     240.0
            LIL Rinascente Torino     860.0
2024-10-11  LIL Rinascente Milano     130.0
2024-10-12  LIL Rinascente Milano     610.0
            LIL Rinascente Torino     200.0
2024-10-13  LIL Rinascente Milano    2440.0
            LIL Rinascente Torino     520.0
2024-10-14  LIL Rinascente Torino     240.

In [4]:
o.to_excel(filepath+"ordini excel.xlsx")

In [ ]:
def handle_pagamenti_methods_diversi(ordini_all, pag):

    # Mask for rows where CHECK is FALSO and Payment Methods contain '+'
    mask = (ordini_all["CHECK"] == "FALSO") & \
        (ordini_all["Payment Method"].str.contains(r'\+', na=False, case=False))
    
    # Get unique names meeting the criteria
    nomi = ordini_all[mask]["Name"].unique()
    
    for n in nomi:
        # Filter rows for this name only
        mask_nomi = ordini_all["Name"] == n
        df_n = ordini_all[mask_nomi]
        
        # If there are 2 or more unique values in 'Numero Pagamento', proceed
        if df_n["Numero Pagamento"].nunique() >= 2:
            # Take the first 'Importo Pagato' value for each unique 'Numero Pagamento' and sum them
            importo_sum = df_n.drop_duplicates(subset="Numero Pagamento")["Importo Pagato"].sum()
            
            # Check if this sum equals the first value in 'Total' for this name
            if importo_sum == df_n["Total"].iloc[0]:
                # Update 'CHECK' to 'VERO' for all rows with this Name
                ordini_all.loc[mask_nomi, "CHECK"] = "VERO"

                # Find matching rows in 'pag' DataFrame where 'Numero Pagamento' is in df_n["Numero Pagamento"]
                mask_pagamenti = pag["Numero Pagamento"].isin(df_n["Numero Pagamento"])
                # Update 'CHECK' column in 'pag' DataFrame for these rows
                pag.loc[mask_pagamenti, "CHECK"] = "VERO"

    return ordini_all, pag

In [ ]:
import pandas as pd

# Sample df_ordini_all DataFrame
data_ordini = {
    "Name": [1, 1, 1, 2, 2, 3, 3],
    "Total": [100, 100, 100, 200, 200, 10, 10],
    "Importo Pagato": [30, 30, 70, 130, 130, 5, 7],
    "Numero Pagamento": ["p1", "p1", "p2", "p3", "p3", "p4", "p5"],
    "CHECK": ["FALSO", "FALSO", "FALSO", "FALSO", "FALSO", "FALSO", "FALSO"],
    "Payment Methods": ["a+b", "a+b", "a+b", "a+c", "a+c", "c+d", "c+d"]
}

df_ordini_all = pd.DataFrame(data_ordini)

# Sample pag DataFrame
data_pag = {
    "Numero Pagamento": ["p1", "p2", "p3", "p4", "p5"],
    "CHECK": ["FALSO", "FALSO", "FALSO", "FALSO", "FALSO"]
}

pag = pd.DataFrame(data_pag)


In [23]:
o['Paid at'] = o['Paid at'].astype(str)

o["Paid at"].dtype

dtype('O')

In [18]:
pd.to_datetime(o['Paid at'], errors='coerce').isna().sum()

273

In [19]:
pd.to_datetime(o2['Paid at'], errors='coerce').isna().sum()

335

In [ ]:
# Convert to datetime, marking unconvertible rows as NaT
o['Converted Date'] = pd.to_datetime(o['Paid at'], errors='coerce')

# Filter rows where conversion failed (NaT values)
problematic_rows = o[o['Converted Date'].isna()]

# Print the problematic rows
print(problematic_rows[['Paid at']])

     Paid at
4        NaN
9        NaN
11       NaN
16       NaN
17       NaN
...      ...
1397     NaN
1398     NaN
1399     NaN
1400     NaN
1401     NaN

[273 rows x 1 columns]


In [30]:
# Remove timezone information
o['Paid at'] = o['Paid at'].str.replace(r'\s[+-]\d{4}$', '', regex=True)
a = pd.to_datetime(o['Paid at'])
a.dtype

dtype('<M8[ns]')

In [ ]:
o["Paid at"].str[:7] 

0       2024-10
1       2024-10
2       2024-10
3       2024-10
4           nan
         ...   
1409    2024-10
1410    2024-10
1411    2024-10
1412    2024-10
1413    2024-10
Name: Paid at, Length: 1414, dtype: object

In [2]:
o = pd.read_csv(filepath+"LIL Milan Orders Export 10.2024.csv") #, thousands='.')
o[o.columns] = o.groupby('Name')[o.columns].ffill()
# o = o[o["Payment Method"].str.contains("PayPal Express Checkout", na=False)]

# o2 = pd.read_csv(filepath+"Ordini LIL.csv") #, thousands='.')
# o2[o2.columns] = o2.groupby('Name')[o2.columns].ffill()
# # o = o[o["Payment Method"].str.contains("PayPal Express Checkout", na=False)]

In [3]:
o["Payment Method"].value_counts()

Payment Method
PayPal Express Checkout                                   484
Shopify Payments                                          344
Custom (POS)                                              121
Scalapay                                                   50
custom                                                     29
Qromo                                                      26
Cash                                                       24
Satispay                                                   15
manual                                                     11
Shopify Payments + PayPal Express Checkout                 11
Shopify Payments + Scalapay                                 9
Gift Card + PayPal Express Checkout                         8
Custom (POS) + manual                                       5
Gift Card + Shopify Payments                                4
Scalapay + Shopify Payments                                 2
Gift Card + Custom (POS)                               

In [4]:
# Define payment methods to exclude
excluded_methods = [
    'Custom (POS)',
    'Qromo ',
    'Custom (POS) + manual',
    'Cash + Custom (POS)',
    "manual",
    "Gift Card + Custom (POS)",
    "Shopify Payments + manual"

]

# Filter out those payment methods
df = o[~o['Payment Method'].isin(excluded_methods)]

# Optional: If you want to see how many were excluded
excluded_count = len(df[df['Payment Method'].isin(excluded_methods)])
df["Payment Method"].value_counts()

df.to_csv(filepath+"ordini 10.csv", index = False)

In [5]:
df["Payment Method"].value_counts()

Payment Method
PayPal Express Checkout                                   484
Shopify Payments                                          344
Scalapay                                                   50
custom                                                     29
Cash                                                       24
Satispay                                                   15
Shopify Payments + PayPal Express Checkout                 11
Shopify Payments + Scalapay                                 9
Gift Card + PayPal Express Checkout                         8
Gift Card + Shopify Payments                                4
Scalapay + Shopify Payments                                 2
Gift Card + Shopify Payments + PayPal Express Checkout      1
Gift Card                                                   1
PayPal Express Checkout + Shopify Payments                  1
Name: count, dtype: int64

In [6]:
df.to_csv(filepath+"ordini_ottobre.csv", index = False)

In [57]:
gift_card_names = df[df["Payment Method"].str.contains("Gift Card") & df["Payment Method"].str.contains(r"\+")]["Name"].unique()
mask = df["Name"].isin(gift_card_names)
print(df.loc[mask, "Payment Method"]) 

5                            Gift Card + Shopify Payments
135                   Gift Card + PayPal Express Checkout
263                   Gift Card + PayPal Express Checkout
395                          Gift Card + Shopify Payments
425     Gift Card + Shopify Payments + PayPal Express ...
472                   Gift Card + PayPal Express Checkout
493                   Gift Card + PayPal Express Checkout
494                   Gift Card + PayPal Express Checkout
495                   Gift Card + PayPal Express Checkout
720                          Gift Card + Shopify Payments
721                          Gift Card + Shopify Payments
802                   Gift Card + PayPal Express Checkout
803                   Gift Card + PayPal Express Checkout
850                              Gift Card + Custom (POS)
1288                             Gift Card + Custom (POS)
Name: Payment Method, dtype: object


In [53]:
for idx, row in df.iterrows():
    if "Gift Card" in str(row["Payment Method"]):
        print(row.Name, row["Payment Method"])

#43014 Gift Card + Shopify Payments
#42927 Gift Card + PayPal Express Checkout
#42833 Gift Card + PayPal Express Checkout
#42745 Gift Card + Shopify Payments
#42726 Gift Card + Shopify Payments + PayPal Express Checkout
#42691 Gift Card + PayPal Express Checkout
#42676 Gift Card + PayPal Express Checkout
#42676 Gift Card + PayPal Express Checkout
#42675 Gift Card + PayPal Express Checkout
#42518 Gift Card + Shopify Payments
#42518 Gift Card + Shopify Payments
#42471 Gift Card + PayPal Express Checkout
#42471 Gift Card + PayPal Express Checkout
#42439 Gift Card + Custom (POS)
#42196 Gift Card
#42174 Gift Card + Custom (POS)


In [46]:
df[df.Name == "#42431"]

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References
861,#42431,federicozagatti@gmail.com,paid,2024-10-18 19:35:25 +0200,fulfilled,2024-10-21 09:07:54 +0200,yes,EUR,167.0,0.0,30.11,167.0,LILGIRL,18.0,NaN,2024-10-18 10:49:03 +0200,1,LIL Bag,5.0,NaN,1.579000e+10,True,True,fulfilled,Federico Zagatti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,manual,#42431.1,0.0,LIL Milan,0.0,Carlotta Trentin,Firgun House,NaN,6.355727e+12,NaN,Low,shopify_draft_order,0.0,IT IVA 22%,30.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#42431.1,Fixed,NaN,#42431.1
862,#42431,federicozagatti@gmail.com,paid,2024-10-18 19:35:25 +0200,fulfilled,2024-10-21 09:07:54 +0200,yes,EUR,167.0,0.0,30.11,167.0,LILGIRL,18.0,NaN,2024-10-18 10:49:03 +0200,1,Primo bacio Necklace - Yellow / 38cm,180.0,NaN,1.579000e+10,True,True,fulfilled,Federico Zagatti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,manual,#42431.1,0.0,LIL Milan,0.0,Carlotta Trentin,Firgun House,NaN,6.355727e+12,NaN,Low,shopify_draft_order,0.0,IT IVA 22%,30.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#42431.1,Fixed,NaN,#42431.1


In [29]:
o[o["Payment Method"] == "Custom (POS)"]

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,Converted Date
10,#43010,matilda.dattilo@gmail.com,paid,2024-10-31 16:37:14 +0100,fulfilled,2024-10-31 16:37:14 +0100,no,EUR,60.0,0.0,0.00,60.0,NaN,0.0,NaN,2024-10-31 16:37:13 +0100,1,E-Gift card - 60.00,60.0,NaN,1.579000e+10,False,False,fulfilled,Matilda Dattilo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Custom (POS),rJBuN1vI5oEBK5lXJqgAWJOUI,0.0,Go Gift Cards,0.0,Veronica Varetta,LIL house,22.0,6.377517e+12,NaN,Low,pos,0.0,IT IVA 0%,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.934768e+11,22-2756,NaN,NaN,NaN,rJBuN1vI5oEBK5lXJqgAWJOUI,NaN,NaN,rJBuN1vI5oEBK5lXJqgAWJOUI,2024-10-31 16:37:14+01:00
12,#43008,gianmarcospadavecchia@protonmail.com,paid,2024-10-31 13:42:03 +0100,fulfilled,2024-10-31 13:42:03 +0100,no,EUR,280.0,0.0,50.49,280.0,NaN,0.0,NaN,2024-10-31 13:42:02 +0100,1,Dna Bracelet - Yellow / 18cm,280.0,NaN,1.579000e+10,True,True,fulfilled,Gianmarco Spadavecchia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Custom (POS),rOxPKGHkn5S2uMWyK6op67vRx,0.0,LIL Milan,0.0,Veronica Varetta,LIL house,22.0,6.377202e+12,NaN,Low,pos,0.0,IT IVA 22%,50.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2755,NaN,NaN,NaN,rOxPKGHkn5S2uMWyK6op67vRx,NaN,NaN,rOxPKGHkn5S2uMWyK6op67vRx,2024-10-31 13:42:03+01:00
14,#43006,gebrayeltony@gmail.com,paid,2024-10-31 13:19:14 +0100,fulfilled,2024-11-01 18:21:04 +0100,no,EUR,420.0,0.0,75.74,420.0,NaN,0.0,NaN,2024-10-31 13:19:14 +0100,1,Tipsy Earcuff Turquoise - Yellow,240.0,NaN,1.579000e+10,True,True,pending,Tony Gebrayel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Custom (POS),rWeRg8xbVvIyEoNRzTOFZI6Aj,0.0,LIL Milan,180.0,Veronica Varetta,LIL house,22.0,6.377133e+12,NaN,Low,pos,0.0,IT IVA 22%,75.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2753,NaN,NaN,NaN,rWeRg8xbVvIyEoNRzTOFZI6Aj,NaN,NaN,rWeRg8xbVvIyEoNRzTOFZI6Aj,2024-10-31 13:19:14+01:00
15,#43006,gebrayeltony@gmail.com,paid,2024-10-31 13:19:14 +0100,fulfilled,2024-11-01 18:21:04 +0100,no,EUR,420.0,0.0,75.74,420.0,NaN,0.0,NaN,2024-10-31 13:19:14 +0100,1,Primo bacio Necklace - Yellow / 38cm,180.0,0.0,1.579000e+10,True,True,fulfilled,Tony Gebrayel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Custom (POS),rWeRg8xbVvIyEoNRzTOFZI6Aj,0.0,LIL Milan,180.0,Veronica Varetta,LIL house,22.0,6.377133e+12,NaN,Low,pos,0.0,IT IVA 22%,75.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2753,NaN,NaN,NaN,rWeRg8xbVvIyEoNRzTOFZI6Aj,NaN,NaN,rWeRg8xbVvIyEoNRzTOFZI6Aj,2024-10-31 13:19:14+01:00
20,#43003,cutronefra@gmail.com,paid,2024-10-31 11:10:26 +0100,fulfilled,2024-10-31 11:10:26 +0100,no,EUR,240.0,0.0,43.28,240.0,NaN,0.0,NaN,2024-10-31 11:10:25 +0100,1,Pensavo fosse amore - Yellow / A,120.0,NaN,1.579000e+10,True,True,fulfilled,Francesco Cutrone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Custom (POS),rwyu19Up6I

In [15]:
o2["Paid at"].dtype

dtype('O')

In [25]:
a = pd.to_datetime(o2['Paid at'])
a.dtype

datetime64[ns, UTC+02:00]

In [3]:
o

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References
0,#43018,gaiapignotti.elly@gmail.com,paid,2024-10-31 23:10:59 +0100,fulfilled,2024-11-03 18:41:31 +0100,yes,EUR,400.00,0.0,72.13,400.00,NaN,0.0,Ups Standard Shipping,2024-10-31 23:10:58 +0100,1,Girls Tears Necklace - White / 39cm,400.00,NaN,1.579000e+10,True,True,fulfilled,Luigina Farronato,Via Oslavia 15,Via Oslavia 15,NaN,NaN,Romano D'Ezzelino,'36060,VI,IT,3925677847,Luigina Farronato,Via Oslavia 15,Via Oslavia 15,NaN,NaN,Romano D'Ezzelino,'36060,VI,IT,3925677847,NaN,lang: en\nInvoice Language: en\nDo you need ou...,NaN,Scalapay,r6cv1e8mOAL89WmnMdvGNmYY0,0.0,LIL Milan,0.00,NaN,NaN,NaN,1.119995e+13,NaN,Low,web,0.0,IT IVA 0%,0.00,IT IVA 22%,72.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vicenza,Vicenza,r6cv1e8mOAL89WmnMdvGNmYY0,NaN,NaN,r6cv1e8mOAL89WmnMdvGNmYY0
1,#43017,frankgordonclark@gmail.com,paid,2024-10-31 21:26:40 +0100,fulfilled,2024-11-02 08:14:27 +0100,no,EUR,393.81,0.0,0.00,393.81,NaN,0.0,NaN,2024-10-31 21:26:39 +0100,2,Glimmer Earring - Yellow / Single,190.94,NaN,1.579000e+10,True,True,fulfilled,Frank Gordon Clark,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Frank gordon Clark,76 Ifield Road,76 Ifield Road,NaN,NaN,London,SW10 9AD,ENG,GB,'+44 7943 389443,NaN,NaN,NaN,custom,rfeJP2N5R7mE4khyZNEoECRBJ,0.0,LIL Milan,-0.01,Veronica Varetta,NaN,NaN,1.119981e+13,NaN,Low,shopify_draft_order,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,England,rfeJP2N5R7mE4khyZNEoECRBJ,NaN,NaN,rfeJP2N5R7mE4khyZNEoECRBJ
2,#43017,frankgordonclark@gmail.com,paid,2024-10-31 21:26:40 +0100,fulfilled,2024-11-02 08:14:27 +0100,no,EUR,393.81,0.0,0.00,393.81,NaN,0.0,NaN,2024-10-31 21:26:39 +0100,1,Luxury Pack + LIL Bag,11.93,NaN,1.579000e+10,True,True,fulfilled,Frank Gordon Clark,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Frank gordon Clark,76 Ifield Road,76 Ifield Road,NaN,NaN,London,SW10 9AD,ENG,GB,'+44 7943 389443,NaN,NaN,NaN,custom,rfeJP2N5R7mE4khyZNEoECRBJ,0.0,LIL Milan,-0.01,Veronica Varetta,NaN,NaN,1.119981e+13,NaN,Low,shopify_draft_order,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,England,rfeJP2N5R7mE4khyZNEoECRBJ,NaN,NaN,rfeJP2N5R7mE4khyZNEoECRBJ
3,#43016,claudiab.burato@gmail.com,paid,2024-10-31 19:14:56 +0100,unfulfilled,NaN,yes,EUR,140.00,10.0,27.05,150.00,NaN,0.0,Ups Standard Shipping,2024-10-31 19:14:56 +0100,1,Rainbow Earring - Yellow / Single / None,140.00,NaN,1.579000e+10,True,True,pending,Claudia Burato,Via Chiesa Volpino 471,Via Chiesa Volpino 471,NaN,NaN,Volpino,'37040,VR,IT,3495814953,Claudia Burato,Via Chiesa Volpino 471,Via Chiesa Volpino 471,NaN,NaN,Volpino,'37040,VR,IT,3495814953,NaN,lang: it\nInvoice Language: it\nDo you need ou...,NaN,PayPal Express Checkout,rDdq7e441xxSByqb28SJjTvBT,0.0,LIL Milan,0.00,NaN,NaN,NaN,6.377734e+12,NaN,Low,web,0.0,IT IVA 22%,27.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Verona,Verona,rDdq7e441xxSByqb28SJjTvBT,NaN,NaN,rDdq7e441xxSByqb28SJjTvBT
4,#43015,NaN,paid,NaN,fulfilled,2024-10-31 18:48:30 +0100,no,EUR,0.00,

In [33]:
p = pd.read_csv(filepath+"Paypal 8 2024.csv")
p = p.groupby('N° ordine commerciante', as_index=False).agg({'Lordo': 'sum',        # Sum the 'Lordo' values
                                                    'Data': 'first',      # Take the first 'Valuta' value for each group
                                                        })

In [165]:
len(o), len(p)

(144, 114)

In [39]:
anno = 2024
mese = 9
f"{anno}-{mese:02}"

'2024-09'

In [37]:
p["Data"].str[3:]

0      08/2024
1      08/2024
2      08/2024
3      08/2024
4      08/2024
        ...   
109    08/2024
110    08/2024
111    08/2024
112    08/2024
113    08/2024
Name: Data, Length: 114, dtype: object

In [34]:
p[p["Data"].str[3:] != "08"]

,N° ordine commerciante,Lordo,Data
0,266225045,"-135,45",24/08/2024
1,402550,"-99,00",14/08/2024
2,r0OK0SNn2YMh1IRz2aofKElmz,"108,00",31/08/2024
3,r0anEzTjKyEpL8eWscCM5nvxo,"252,00",22/08/2024
4,r0lp3CTOVzkicI2BUPqtpqUQW,"600,00",15/08/2024
...,...,...,...
109,ry8roP7yLQi7zqWbVcmUATZMM,"171,00",29/08/2024
110,ryPVdYeEUWbpwx3lfWro8N1uk,"216,00",14/08/2024
111,rycgeT6vlpqrzJE21HBD2ZCNz,"270,00",04/08/2024
112,ryfExMry2G0NS7TQ9vU9ZwCOx,"5,00",07/08/2024


In [166]:
merge = pd.merge(o, p, left_on="Payment Reference", right_on="N° ordine commerciante", how = "outer")

In [168]:
merge[merge.Name.isna()]

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,N° ordine commerciante,Lordo,Data
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,266225045,"-135,45",24/08/2024
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,402550,"-99,00",14/08/2024
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,r0OK0SNn2YMh1IRz2aofKElmz,"108,00",31/08/2024
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,r1K2A8LYKpITAyctTq68mux2D,"102,00",29/08/2024
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,r4T9aJ5Mk5VN5h95fJI3HaRcY,"160,00",13/08/2024
27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rBR2DZRyaRO0Kgi6ufbJlZW8u,"100,00",30/08/2024
43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rHOGsgs8N9VNgueWzI1js6pZt,"135,00",28/08/2024
47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rLqsoy16vTUOmsXSTfKnwuDgg,"150,00",24/0

In [155]:
sh = pd.read_csv(filepath+"Shopify LIL 8 2024.csv")
sh = sh.groupby('Order', as_index=False).agg({'Amount': 'sum',        # Sum the 'Lordo' values
                                                    'Transaction Date': 'first',      # Take the first 'Valuta' value for each group
                                                        })

In [156]:
# Assuming 'date_column' is the name of the column containing datetime values
cutoff_date = '2024-08-27 11:42:28'
sh = sh[sh['Transaction Date'] <= cutoff_date]


In [157]:
len(o), len(sh)

(133, 84)

In [158]:
o[o.Name=="#41324"]

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References
89,#41324,jennifer.righi25@gmail.com,paid,2024-08-21 14:29:56 +0200,fulfilled,2024-08-22 12:20:39 +0200,yes,EUR,360.0,0.0,64.92,360.0,NaN,0.0,Ups Standard Shipping,2024-08-21 14:29:55 +0200,1,Breeze - Yellow / 60cm,260.0,NaN,1.579000e+10,True,True,fulfilled,Lorenzo Papi,Via Castello 4,Via Castello 4,NaN,NaN,Pietrarubbia,'61023,PU,IT,+393667278131,Lorenzo Papi,Via Castello 4,Via Castello 4,NaN,NaN,Pietrarubbia,'61023,PU,IT,+393667278131,NaN,lang: it\nInvoice Language: it\nDo you need ou...,NaN,Scalapay + Shopify Payments,r5ihysXwP1tMlbrsEizt8CZCM,0.0,LIL Milan,0.0,NaN,NaN,NaN,6.269494e+12,NaN,Low,web,0.0,IT IVA 22%,64.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pesaro and Urbino,Pesaro and Urbino,r5ihysXwP1tMlbrsEizt8CZCM,NaN,NaN,r97vRu6iEbQgI79Jc1FXoabp8 + r5ihysXwP1tMlbrsEi...
90,#41324,jennifer.righi25@gmail.com,paid,2024-08-21 14:29:56 +0200,fulfilled,2024-08-22 12:20:39 +0200,yes,EUR,360.0,0.0,64.92,360.0,NaN,0.0,Ups Standard Shipping,2024-08-21 14:29:55 +0200,1,Pensavo fosse amore - Yellow / J,100.0,NaN,1.579000e+10,True,True,fulfilled,Lorenzo Papi,Via Castello 4,Via Castello 4,NaN,NaN,Pietrarubbia,'61023,PU,IT,+393667278131,Lorenzo Papi,Via Castello 4,Via Castello 4,NaN,NaN,Pietrarubbia,'61023,PU,IT,+393667278131,NaN,lang: it\nInvoice Language: it\nDo you need ou...,NaN,Scalapay + Shopify Payments,r5ihysXwP1tMlbrsEizt8CZCM,0.0,LIL Milan,0.0,NaN,NaN,NaN,6.269494e+12,NaN,Low,web,0.0,IT IVA 22%,64.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pesaro and Urbino,Pesaro and Urbino,r5ihysXwP1tMlbrsEizt8CZCM,NaN,NaN,r97vRu6iEbQgI79Jc1FXoabp8 + r5ihysXwP1tMlbrsEi...


In [159]:
merge = pd.merge(o, sh, left_on="Name", right_on="Order", how = "outer")

In [160]:
display(merge[merge.Name.isna()])

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,Order,Amount,Transaction Date
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#40895,80.0,2024-08-05 22:15:32 +0200
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#40906,20.0,2024-08-16 17:23:20 +0200
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,#41019,-322.2,2024-08-01 10:09:42 +0200


In [58]:
q = pd.read_csv(filepath+"qromo.csv", thousands='.')
q["Data"].dtype

dtype('O')

In [59]:
sc = pd.read_csv(filepath+"scalapay 9.2023.csv")
sc["Data acquisto/rimborso"].dtype

dtype('O')

In [132]:
sh = pd.read_csv(filepath+"Shopify LIL 8 2024.csv")
sh["Transaction Date"].dtype

dtype('O')

In [61]:
p = pd.read_csv(filepath+"Paypal.csv", thousands='.')
p["Data"].dtype

dtype('O')

In [65]:
b = pd.read_excel(filepath+"Intesa 9.xlsx", dtype={"Data": str}, skiprows= 19)
b["Data"].dtype

dtype('O')

In [4]:
sa = pd.read_csv(filepath+"Satispay LIL Online 10.2024.csv")
sa["payment_date"].dtype

dtype('O')

In [5]:
sa

,payment_uid,payment_date,shop_name,cost_centre,total_amount,fee_amount,payment_type,description,source_payment_uid,satispay_amount,fringe_amount
0,eca036ee-feef-4e94-8297-fbd71b925d6c,2024-10-27 18:26:04.116,lil milan,0,208.0,2.28,TO_BUSINESS,rmygoylCsqZOkRJvS8HrRkPay,0,208.0,0.0
1,8d7782e7-afd8-4ea3-900b-dad152217305,2024-10-26 21:22:30.543,lil milan,0,224.0,2.44,TO_BUSINESS,rWKRNz959OrCjvPRLNECUD1dg,0,224.0,0.0
2,d2adf40d-2ef0-4334-a6c5-8691e1e4a549,2024-10-26 18:58:32.094,lil milan,0,208.0,2.28,TO_BUSINESS,rrS7ECF0HiVPvtQluYHgDaOus,0,208.0,0.0
3,adb761b3-36cb-4791-af20-6e0bbb235072,2024-10-26 17:19:17.273,lil milan,0,608.0,6.28,TO_BUSINESS,rYewzxnUhUk0Olbgj8N5Og2vE,0,608.0,0.0
4,f4ab3ced-59c5-4677-9f6f-0d2810ef05ff,2024-10-26 13:44:18.826,lil milan,0,320.0,3.40,TO_BUSINESS,rtFxinUKsiw0UakmOPGVeqT27,0,320.0,0.0
5,fac96b64-e56e-4060-a185-6f983c1b113e,2024-10-26 09:25:54.255,lil milan,0,256.0,2.76,TO_BUSINESS,rbuCJmOh7CXtySVoRhrv3LSAp,0,256.0,0.0
6,714b1533-6c10-4dc7-b885-cd42af5a77df,2024-10-17 13:04:02.988,lil milan,0,130.0,1.50,TO_BUSINESS,rse9kycxhE0glrrwYKwjIGV7z,0,130.0,0.0
7,3767b465-dc61-4a78-9a69-6aa6662d93ac,2024-10-17 12:39:41.618,lil milan,0,30.0,0.50,TO_BUSINESS,rgL3oiQNFw8LwdKDh9CGsRomK,0,30.0,0.0
8,d5784444-35f3-4a6f-9830-010745f7d3b2,2024-10-14 15:16:19.805,lil milan,0,30.0,0.50,TO_BUSINESS,r1H9JKkHesXvswcoZsLZCQ6rf,0,30.0,0.0
9,a599294a-0cbd-4bbf-bca9-f4f1a9a8e854,2024-10-14 15:15:01.565,lil milan,0,30.0,0.50,TO_BUSINESS,rdm22QbhNBZd1Rz1wYnKeZCEb,0,30.0,0.0


In [6]:
sa = pd.read_csv(filepath+"Satispay.csv")
sa["payment_date"].dtype

dtype('O')

In [7]:
sa

,payment_uid,payment_date,shop_name,cost_centre,total_amount,fee_amount,payment_type,description,source_payment_uid,satispay_amount,fringe_amount
0,477db8e5-b3a3-47ed-bb1a-21bb6fbe2aef,2024-09-30 11:01:47.309,lil milan,0,100.0,1.20,TO_BUSINESS,rv5MJ5MXPdxE6fke5gpa9misQ,0,100.0,0.0
1,12c2a6c9-7d77-4e1f-b7a3-6198259d3e18,2024-09-29 22:47:47.016,lil milan,0,70.0,0.90,TO_BUSINESS,rvglWnJJGxy1xunSP7DKQmWHP,0,70.0,0.0
2,966682fd-2bb2-4ca5-9881-5b7f7b8b8e2d,2024-09-27 23:09:42.994,lil milan,0,30.0,0.50,TO_BUSINESS,r0RZ50RjLdHxdIKvcoRh1bkGh,0,30.0,0.0
3,3ed542f0-135b-4cf0-ac4b-36329cd5de8b,2024-09-21 18:39:09.615,lil milan,0,10.0,0.00,TO_BUSINESS,0,0,10.0,0.0
4,e6f50038-b810-4d42-af2f-557714cf4d4f,2024-09-20 18:33:03.284,lil milan,0,112.0,1.32,TO_BUSINESS,ruO1LIJmVe82x9nj5FhDsBVg5,0,112.0,0.0
5,8dbd3c29-bc47-4248-94b6-dbb886754b9e,2024-09-18 17:23:30.175,lil milan,0,100.0,1.20,TO_BUSINESS,rRH4BwjlFn2DGwLWZHNwDUnqI,0,100.0,0.0
6,d21d509c-856d-499f-a527-bdfd9ebb64d0,2024-09-14 17:13:45.485,lil milan,0,200.0,0.20,TO_BUSINESS,0,0,200.0,0.0
7,2f28b676-0d55-4589-a1e4-2930184361aa,2024-09-13 14:22:46.551,lil milan,0,200.0,0.20,TO_BUSINESS,0,0,200.0,0.0
8,abb1d9f8-b03a-40f2-bde1-d796cf5ce71c,2024-09-05 13:24:01.176,lil milan,0,30.0,0.50,TO_BUSINESS,rGCOQAYdXXoh5MRcgV4z6eSnT,0,30.0,0.0
9,2b59995e-e371-45d4-8f97-859c54069943,2024-09-04 17:50:21.384,lil milan,0,82.0,1.02,TO_BUSINESS,r5d3JSnOvjqAYen4kKUGlD2jg,0,82.0,0.0


In [71]:
p.loc[0,"Data"]

'01/09/2024'

In [106]:
o["Paid at"] = o.groupby('Name')["Paid at"].ffill()

In [107]:
for i in range(1023, 1029):
    if i not in o.index:
        o.loc[i] = [None] * len(o.columns)  # Initialize with None or NaN
    # 

In [108]:
o.iloc[1023, 3] = p.loc[0, "Data"]
o.loc[1023, "Payment Method"] = "PayPal Express Checkout"

o.iloc[1024, 3] = q.loc[0, "Data"]
o.loc[1024, "Payment Method"] = "Qromo"

o.iloc[1025, 3] = b.loc[0, "Data"]
o.loc[1025, "Payment Method"] = "Bonifico"

o.iloc[1026, 3] = sa.loc[0, "payment_date"]
o.loc[1026, "Payment Method"] = "Satispay"

o.iloc[1027, 3] = sc.loc[0, "Data acquisto/rimborso"]
o.loc[1027, "Payment Method"] = "Scalapay"

o.iloc[1028, 3] = sh.loc[0, "Transaction Date"]
o.loc[1028, "Payment Method"] = "Shopify Payments"

In [110]:
o.tail(20)

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,3
1009,#41475,lauralombardi@hotmail.it,paid,NaN,fulfilled,2024-09-01 19:39:23 +0200,no,EUR,0.0,0.0,0.00,0.0,MILANO100%,100.0,NaN,2024-09-01 19:39:22 +0200,1.0,Pensavo fosse amore - Yellow / B,100.0,NaN,1.579000e+10,True,True,fulfilled,Laura Lombardi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ST,NaN,NaN,NaN,NaN,0.0,LIL Milan,0.0,Veronica Varetta,LIL Rinascente Milano,23.0,6.284748e+12,NaN,Low,pos,0.0,IT IVA 22%,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23-2371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1010,#41474,alebaietto98@gmail.com,paid,NaN,fulfilled,2024-09-01 16:57:50 +0200,no,EUR,0.0,0.0,0.00,0.0,TORINO100%,260.0,NaN,2024-09-01 16:57:50 +0200,1.0,Goldie Hoop - Yellow / Single,260.0,NaN,1.579000e+10,True,True,fulfilled,Alessia Baietto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GdM,NaN,NaN,NaN,NaN,0.0,LIL Milan,0.0,Veronica Varetta,LIL Rinascente Torino,3.0,6.284531e+12,NaN,Low,pos,0.0,IT IVA 22%,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.936693e+11,3-5679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1011,#41473,alessandradim@live.it,paid,2024-09-01 16:51:44 +0200,fulfilled,2024-09-02 09:38:23 +0200,yes,EUR,270.0,0.0,48.69,270.0,BACK10,30.0,Ups Standard Shipping,2024-09-01 16:51:43 +0200,1.0,Boys Tears - Yellow / 35cm,300.0,NaN,1.579000e+10,True,True,fulfilled,Alessandra Di mauro,Via Callipoli 228,Via Callipoli 228,NaN,NaN,Giarre,'95014,CT,IT,NaN,Francesco Longhitano,"Via Ferrarese 219, Primo piano NEXT srl",Via Ferrarese 219,Primo piano NEXT srl,NEXT srl,Bologna,'40128,BO,IT,+393293126395,NaN,lang: it\nInvoice Language: it\nDo you need ou...,NaN,PayPal Express Checkout,r6PQK1hUrsM3aRqKMpsC4TCMp,0.0,LIL Milan,0.0,NaN,NaN,NaN,6.284523e+12,NaN,Low,web,0.0,IT IVA 22%,48.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Catania,Bologna,r6PQK1hUrsM3aRqKMpsC4TCMp,NaN,NaN,r6PQK1hUrsM3aRqKMpsC4TCMp,NaN
1012,#41472,viola_fiore@outlook.it,paid,NaN,fulfilled,2024-09-01 15:26:12 +0200,no,EUR,0.0,0.0,0.00,0.0,TORINO100%,460.0,NaN,2024-09-01 15:26:11 +0200,1.0,11 / I am Honey - Yellow,260.0,NaN,1.579000e+10,True,True,fulfilled,Viola Fioravante,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GdM,NaN,NaN,NaN,NaN,0.0,LIL Milan,0.0,Veronica Varetta,LIL Rinascente Torino,3.0,6.284406e+12,NaN,Low,pos,0.0,IT IVA 22%,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3-5678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1013,#41472,viola_fiore@outlook.it,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-09-01 15:26:11 +0200,1.0,LIL Extender - Yellow,60.0,NaN,1.579000e+10,True,True,fulfilled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIL Milan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1014,#41472,viola_fiore@outlook.it,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-09-01 15:26:11 +0200,1.0

In [122]:
# Function to check and reformat
def reformat_date(date_str):
    if pd.notna(date_str):
        date_str = date_str.strip().replace("/", "-")[:10]
    # Check if the string starts with "2024"
        if not date_str.startswith("2024"):
            # Split and rearrange as "YYYY-MM-DD"
            return "-".join(date_str.split("-")[::-1])
    return date_str  # Return as-is if already starts with "2024"

# Apply the function to each row in the column
o['partial_date'] = o['Paid at'].apply(reformat_date)

In [123]:
o.tail(20)

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,3,partial_date
1009,#41475,lauralombardi@hotmail.it,paid,NaN,fulfilled,2024-09-01 19:39:23 +0200,no,EUR,0.0,0.0,0.00,0.0,MILANO100%,100.0,NaN,2024-09-01 19:39:22 +0200,1.0,Pensavo fosse amore - Yellow / B,100.0,NaN,1.579000e+10,True,True,fulfilled,Laura Lombardi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ST,NaN,NaN,NaN,NaN,0.0,LIL Milan,0.0,Veronica Varetta,LIL Rinascente Milano,23.0,6.284748e+12,NaN,Low,pos,0.0,IT IVA 22%,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23-2371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1010,#41474,alebaietto98@gmail.com,paid,NaN,fulfilled,2024-09-01 16:57:50 +0200,no,EUR,0.0,0.0,0.00,0.0,TORINO100%,260.0,NaN,2024-09-01 16:57:50 +0200,1.0,Goldie Hoop - Yellow / Single,260.0,NaN,1.579000e+10,True,True,fulfilled,Alessia Baietto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GdM,NaN,NaN,NaN,NaN,0.0,LIL Milan,0.0,Veronica Varetta,LIL Rinascente Torino,3.0,6.284531e+12,NaN,Low,pos,0.0,IT IVA 22%,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.936693e+11,3-5679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1011,#41473,alessandradim@live.it,paid,2024-09-01 16:51:44 +0200,fulfilled,2024-09-02 09:38:23 +0200,yes,EUR,270.0,0.0,48.69,270.0,BACK10,30.0,Ups Standard Shipping,2024-09-01 16:51:43 +0200,1.0,Boys Tears - Yellow / 35cm,300.0,NaN,1.579000e+10,True,True,fulfilled,Alessandra Di mauro,Via Callipoli 228,Via Callipoli 228,NaN,NaN,Giarre,'95014,CT,IT,NaN,Francesco Longhitano,"Via Ferrarese 219, Primo piano NEXT srl",Via Ferrarese 219,Primo piano NEXT srl,NEXT srl,Bologna,'40128,BO,IT,+393293126395,NaN,lang: it\nInvoice Language: it\nDo you need ou...,NaN,PayPal Express Checkout,r6PQK1hUrsM3aRqKMpsC4TCMp,0.0,LIL Milan,0.0,NaN,NaN,NaN,6.284523e+12,NaN,Low,web,0.0,IT IVA 22%,48.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Catania,Bologna,r6PQK1hUrsM3aRqKMpsC4TCMp,NaN,NaN,r6PQK1hUrsM3aRqKMpsC4TCMp,NaN,2024-09-01
1012,#41472,viola_fiore@outlook.it,paid,NaN,fulfilled,2024-09-01 15:26:12 +0200,no,EUR,0.0,0.0,0.00,0.0,TORINO100%,460.0,NaN,2024-09-01 15:26:11 +0200,1.0,11 / I am Honey - Yellow,260.0,NaN,1.579000e+10,True,True,fulfilled,Viola Fioravante,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GdM,NaN,NaN,NaN,NaN,0.0,LIL Milan,0.0,Veronica Varetta,LIL Rinascente Torino,3.0,6.284406e+12,NaN,Low,pos,0.0,IT IVA 22%,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3-5678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1013,#41472,viola_fiore@outlook.it,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-09-01 15:26:11 +0200,1.0,LIL Extender - Yellow,60.0,NaN,1.579000e+10,True,True,fulfilled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LIL Milan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1014,#41472,viola_fiore@outlook.it,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [55]:
q = pd.read_excel(filepath+"output_file (11).xlsx", sheet_name = "Ordini LIL")
q["Paid at"].dtype

dtype('O')

In [22]:
s["Stato"].value_counts()

Stato
Riuscito      124
Annullato      14
Rimborsato      1
Name: count, dtype: int64

In [38]:
s = s[s["Stato"] != "Annullato"]
len(s)

125

In [32]:
s["Importo €"] = s["Importo €"].astype(float)
s["Importo rimborsato €"] = s["Importo rimborsato €"].astype(float)
s["Importo Pagato"] = s["Importo €"] - s["Importo rimborsato €"]

,Data,Stato,Importo €,Importo rimborsato €,ID,Importo Pagato
5,2024-09-28 19:05:04,Riuscito,260.0,0.0,pi_3Q444qL1BrR61arW10hQ69ks,260.0
6,2024-09-28 18:22:01,Riuscito,80.0,0.0,pi_3Q43PBL1BrR61arW1ZnM6W3P,80.0
7,2024-09-28 18:20:32,Riuscito,198.0,0.0,pi_3Q43NkL1BrR61arW1BFhBCNZ,198.0
8,2024-09-28 18:16:38,Riuscito,248.0,0.0,pi_3Q43JyL1BrR61arW0JQC05IY,248.0
11,2024-09-28 18:14:06,Riuscito,238.0,0.0,pi_3Q43HWL1BrR61arW0S8K5pqk,238.0
12,2024-09-28 17:11:46,Riuscito,190.0,0.0,pi_3Q42JCL1BrR61arW0jOP3kgi,190.0
13,2024-09-28 16:35:46,Riuscito,516.0,0.0,pi_3Q41kML1BrR61arW16JpGdeL,516.0
14,2024-09-28 16:26:58,Riuscito,40.0,0.0,pi_3Q41bqL1BrR61arW1VDD4zXb,40.0
15,2024-09-28 16:08:29,Riuscito,98.0,0.0,pi_3Q41JxL1BrR61arW0OK0QGKs,98.0
16,2024-09-28 15:58:55,Riuscito,580.0,0.0,pi_3Q41AhL1BrR61arW0uCqlfsw,580.0


In [37]:
s["Importo Pagato"].sum()

24288.0

In [6]:
agosto[agosto.ID.isna()]

,Data,Stato,Importo €,Importo rimborsato €,ID


In [2]:
#ordini
ordini_processor = Ordini(filepath, mese = 8)
ordini = ordini_processor.preprocess()

#run matchers
shopify_matcher = ShopifyMatcher(filepath, df_ordini=ordini)
scalapay_matcher = ScalapayMatcher(filepath, df_ordini=ordini)
satispay_matcher = SatispayMatcher(filepath, df_ordini=ordini)
paypal_matcher = PaypalMatcher(filepath, df_ordini=ordini)
qromo_matcher = QromoMatcher(filepath, df_ordini=ordini)
bonifico_matcher = BonificoMatcher(filepath, df_ordini=ordini)

# Create the matchers list
matchers = [
    shopify_matcher,
    scalapay_matcher,
    satispay_matcher,
    paypal_matcher,
    qromo_matcher,
    bonifico_matcher
]

#excel
runner = MatcherRunner(matchers, ordini)
df_check_all, df_ordini_all, df = runner.run_all_matchers(mese=8, anno= 2024)

Non ci sono pagamenti con Satispay
All 'Names' have consistent 'Total' values.



'LIL Milan'

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,Unnamed: 79,COGS,Tot,COGS Packaging,Tot.1,Brand,Order,Amount,CHECK,CHECK_VALORE_GIFT_CARD
5,#41019,NaN,refunded,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,Boys Tears - Yellow / 35cm,NaN,NaN,1.579000e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IT,NaN,NaN,NaN,NaN,Shopify Payments,NaN,322.2,NaN,NaN,NaN,Firgun House,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.845,-98.845,2.36,-2.36,LIL Milan,#41019,-322.2,NON TROVATO,NaN


In [3]:
#ordini
ordini_processor = Ordini(filepath, mese = 9)
ordini = ordini_processor.preprocess()

#run matchers
shopify_matcher = ShopifyMatcher(filepath, df_ordini=ordini)
scalapay_matcher = ScalapayMatcher(filepath, df_ordini=ordini)
satispay_matcher = SatispayMatcher(filepath, df_ordini=ordini)
paypal_matcher = PaypalMatcher(filepath, df_ordini=ordini)
qromo_matcher = QromoMatcher(filepath, df_ordini=ordini)
bonifico_matcher = BonificoMatcher(filepath, df_ordini=ordini)

# Create the matchers list
matchers = [
    shopify_matcher,
    scalapay_matcher,
    satispay_matcher,
    paypal_matcher,
    qromo_matcher,
    bonifico_matcher
]

#excel
runner = MatcherRunner(matchers, ordini)
df_check_all, df_ordini_all, df = runner.run_all_matchers(mese=9, anno= 2024)

All 'Names' have consistent 'Total' values.



'LIL Milan'

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,Brand,Order,Amount,CHECK,CHECK_VALORE_GIFT_CARD
91,#41676,laure.michelwatrin@outlook.com,paid,2024-09-11 20:26:58 +0200,fulfilled,2024-09-12 08:41:08 +0200,yes,EUR,120.0,0.0,20.00,120.0,NaN,0.0,UPS Standard International,2024-09-11 20:26:57 +0200,1.0,Boys Tears Ring - Yellow / 15,120.0,NaN,015790001403,True,True,fulfilled,Laure Michel-Watrin,13 Rue Commandant Sigoyer,13 Rue Commandant Sigoyer,NaN,NaN,Thionville,'57100,NaN,FR,0787024875,Laure Michel-Watrin,13 Rue Commandant Sigoyer,13 Rue Commandant Sigoyer,NaN,NaN,Thionville,'57100,NaN,FR,0787024875,NaN,lang: en\nInvoice Language: en\nDo you need ou...,NaN,Gift Card + Shopify Payments,rUACPOx5Q2ga6XLtZxRipWurH,0.0,LIL Milan,0.0,NaN,Firgun House,<NA>,6300226486621,<NA>,Low,web,0.0,FR TVA 20%,20.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,raXmOKggE46HEwSvGNV7jccqK + rUACPOx5Q2ga6XLtZx...,NaN,<NA>,raXmOKggE46HEwSvGNV7jccqK + rUACPOx5Q2ga6XLtZx...,LIL Milan,#41676,45.0,FALSO,75.0
102,#41700,grethe@hotmail.it,paid,2024-09-13 09:52:32 +0200,fulfilled,2024-09-16 08:58:03 +0200,yes,EUR,90.0,0.0,16.23,90.0,LILGIRL,10.0,Ups Standard Shipping,2024-09-13 09:52:32 +0200,1.0,Girls Tears Ring - Yellow / 11,100.0,NaN,015790000954,True,True,fulfilled,Grethe Nielsen,Via Leonardo Da Vinci 47,Via Leonardo Da Vinci 47,NaN,NaN,Como,'22100,CO,IT,3493303536,Grethe Nielsen,Via Leonardo Da Vinci 47,Via Leonardo Da Vinci 47,NaN,NaN,Como,'22100,CO,IT,3493303536,NaN,lang: it\nInvoice Language: it\nDo you need ou...,NaN,Gift Card + Shopify Payments,rne6FVyV32soTd0LngZP4uZSA,0.0,LIL Milan,0.0,NaN,Firgun House,<NA>,6303730893149,<NA>,Low,web,0.0,IT IVA 22%,16.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Como,Como,rB4xrbWD88ISTHqqh3j2LPjJF + rne6FVyV32soTd0Lng...,NaN,<NA>,rB4xrbWD88ISTHqqh3j2LPjJF + rne6FVyV32soTd0Lng...,LIL Milan,#41700,40.0,FALSO,50.0
210,#41959,serenapiavani@gmail.com,paid,2024-09-25 00:01:02 +0200,unfulfilled,NaN,yes,EUR,270.0,0.0,48.69,270.0,NaN,0.0,Ups Standard Shipping,2024-09-25 00:01:01 +0200,1.0,Sweet Spot - Yellow / matte / White,260.0,NaN,015790000015,True,True,pending,Serena Piavani,Vicolo Michele Ponzetti 22,Vicolo Michele Ponzetti 22,NaN,NaN,Romano di Lombardia,'24058,BG,IT,3471668869,Serena Piavani,Vicolo Michele Ponzetti 22,Vicolo Michele Ponzetti 22,NaN,NaN,Romano di Lombardia,'24058,BG,IT,3471668869,NaN,lang: it\nInvoice Language: it\nDo you need ou...,NaN,Gift Card + Shopify Payments,r6k9ibaeVXculPTCxTmXrZ7m4,0.0,LIL Milan,0.0,NaN,Firgun House,<NA>,6319286976861,<NA>,Low,web,0.0,IT IVA 22%,48.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bergamo,Bergamo,rEo4hGP9HCaRpJ288Wy7I67Zp + r6k9ibaeVXculPTCxT...,NaN,<NA>,rEo4hGP9HCaRpJ288Wy7I67Zp + r6k9ibaeVXculPTCxT...,LIL Milan,#41959,20.0,FALSO,250.0
211,#41959,serenapiavani@gmail.com,paid,2024-09-25 00:01:02 +0200,unfulfilled,NaN,yes,EUR,270.0,0.0,48.69,270.0,NaN,0.0,Ups Standard Shipping,2024-09-25 00:01:01 +0200,1.0,Engraving,10.0,NaN,01579000

'LIL Milan'

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,Brand,Matched Reference,Merchant ID,Import lordo,CHECK,CHECK_VALORE_GIFT_CARD
5,#41790,la_cami@hotmail.it,paid,2024-09-17 16:20:06 +0200,fulfilled,2024-09-18 10:02:38 +0200,yes,EUR,180.0,0.0,32.46,180.0,BACK10,20.0,Ups Standard Shipping,2024-09-17 16:20:06 +0200,1.0,Boys Tears Ring - Yellow / 12,120.0,NaN,015790001402,True,True,fulfilled,Laura Camilotto,"Via Piva, 59/A",Via Piva,59/A,c/o SECAM SRL,Valdobbiadene,'31049,TV,IT,+393381460723,Laura Camilotto,"Via Piva, 59/A",Via Piva,59/A,c/o SECAM SRL,Valdobbiadene,'31049,TV,IT,+393381460723,NaN,lang: en\nInvoice Language: en\nDo you need ou...,NaN,Scalapay + Gift Card,rtPaaIuJs2HypP9OSedzVDHll,0.0,LIL Milan,0.0,NaN,Firgun House,<NA>,6309634113885,<NA>,Low,web,0.0,IT IVA 22%,32.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.933815e+11,NaN,NaN,Treviso,Treviso,r5jeAybN2TGr9cvkk5PI5MWj8 + rtPaaIuJs2HypP9OSe...,NaN,<NA>,r5jeAybN2TGr9cvkk5PI5MWj8 + rtPaaIuJs2HypP9OSe...,LIL Milan,r5jeAybN2TGr9cvkk5PI5MWj8,r5jeAybN2TGr9cvkk5PI5MWj8,175.0,FALSO,5.0
6,#41790,la_cami@hotmail.it,paid,2024-09-17 16:20:06 +0200,fulfilled,2024-09-18 10:02:38 +0200,yes,EUR,180.0,0.0,32.46,180.0,BACK10,20.0,Ups Standard Shipping,2024-09-17 16:20:06 +0200,1.0,Nude Ring - White / 10,80.0,NaN,015790000224,True,True,fulfilled,Laura Camilotto,"Via Piva, 59/A",Via Piva,59/A,c/o SECAM SRL,Valdobbiadene,'31049,TV,IT,+393381460723,Laura Camilotto,"Via Piva, 59/A",Via Piva,59/A,c/o SECAM SRL,Valdobbiadene,'31049,TV,IT,+393381460723,NaN,lang: en\nInvoice Language: en\nDo you need ou...,NaN,Scalapay + Gift Card,rtPaaIuJs2HypP9OSedzVDHll,0.0,LIL Milan,0.0,NaN,Firgun House,<NA>,6309634113885,<NA>,Low,web,0.0,IT IVA 22%,32.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.933815e+11,NaN,NaN,Treviso,Treviso,r5jeAybN2TGr9cvkk5PI5MWj8 + rtPaaIuJs2HypP9OSe...,NaN,<NA>,r5jeAybN2TGr9cvkk5PI5MWj8 + rtPaaIuJs2HypP9OSe...,LIL Milan,r5jeAybN2TGr9cvkk5PI5MWj8,r5jeAybN2TGr9cvkk5PI5MWj8,175.0,FALSO,5.0


...

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,Brand,Matched Reference,N° ordine commerciante,Lordo,Valuta,CHECK,CHECK_VALORE_GIFT_CARD
44,#41609,carolaferrari7@gmail.com,paid,2024-09-09 09:35:13 +0200,fulfilled,2024-09-10 08:28:34 +0200,yes,EUR,160.0,0.0,28.85,160.0,NaN,0.0,Ups Standard Shipping,2024-09-09 09:35:13 +0200,1.0,Limitless Earring - Yellow / Single,160.0,NaN,015790000056,True,True,fulfilled,Carola Ferrari,"Via Poste 1A, Campanello: Ferrari Carola/ Lona...",Via Poste 1A,Campanello: Ferrari Carola/ Lonardi Andrea,NaN,Marano di Valpolicella,'37020,VR,IT,346 186 9399,Carola Ferrari,"Via Tobele, 6, Campanello: Ferrari Diego","Via Tobele, 6",Campanello: Ferrari Diego,NaN,Marano Di Valpolicella,'37020,VR,IT,+393461869399,NaN,lang: it\nInvoice Language: it\nDo you need ou...,NaN,Gift Card + PayPal Express Checkout,rAs7nyWDL1TpV8mUpBw2oDBp3,0.0,LIL Milan,0.0,NaN,Firgun House,<NA>,6294665068893,<NA>,Low,web,0.0,IT IVA 22%,28.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Verona,Verona,rAZJD4nM7ossbkbUTK6y2xaXF + rAs7nyWDL1TpV8mUpB...,NaN,<NA>,rAZJD4nM7ossbkbUTK6y2xaXF + rAs7nyWDL1TpV8mUpB...,LIL Milan,rAs7nyWDL1TpV8mUpBw2oDBp3,rAs7nyWDL1TpV8mUpBw2oDBp3,60.0,EUR,FALSO,100.0
139,#41556,martinautenze@gmail.com,paid,2024-09-06 16:27:22 +0200,fulfilled,2024-09-10 13:10:32 +0200,yes,EUR,300.0,0.0,54.10,300.0,NaN,0.0,Firgun House,2024-09-06 16:27:21 +0200,1.0,Boys Tears Necklace - Yellow / 35cm,300.0,NaN,015790000008,True,True,fulfilled,Martina Chiri,Via Scipione Pistrucci 4,Via Scipione Pistrucci 4,NaN,NaN,Milano,'20137,MI,IT,3458107910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IT,NaN,NaN,lang: it\nInvoice Language: it\nDo you need ou...,NaN,Gift Card + PayPal Express Checkout,rYRG5HrMJdrRjAOTPb74bH09C,0.0,LIL Milan,0.0,NaN,Firgun House,<NA>,6291389907293,<NA>,Low,web,0.0,IT IVA 22%,54.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Milan,NaN,rVOXPhjTEQwUbzlmt6dBME1f3 + rs7Eet0cepmwF8JoSv...,NaN,<NA>,rVOXPhjTEQwUbzlmt6dBME1f3 + rs7Eet0cepmwF8JoSv...,LIL Milan,rYRG5HrMJdrRjAOTPb74bH09C,rYRG5HrMJdrRjAOTPb74bH09C,12.0,EUR,FALSO,288.0


'LIL Milan'

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,Brand,Paid_datetime,Stato,Importo €,Importo rimborsato €,Importo Effettivo,Data_datetime,CHECK,CHECK_VALORE_GIFT_CARD
30,#41574,carla.difranco01@gmail.com,paid,2024-09-07 13:06:24 +0200,fulfilled,2024-09-07 13:05:43 +0200,no,EUR,360.0,0.0,64.92,360.0,NaN,0.0,NaN,2024-09-07 13:05:43 +0200,1,Girls Tears Necklace - Yellow / 39cm,360.0,NaN,015790000834,True,True,fulfilled,Carla Di franco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IT,NaN,NaN,NaN,NaN,Gift Card + Qromo,rfOtIzWbrhWAResOHedxpeyoE,0.0,LIL Milan,0.0,Veronica Varetta,LIL House,22,6292369572189,<NA>,Low,pos,0.0,IT IVA 22%,64.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2475,NaN,NaN,NaN,rQtzuqGxdvZh6i29klr1R646G + rfOtIzWbrhWAResOHe...,Due on receipt,<NA>,rQtzuqGxdvZh6i29klr1R646G + rfOtIzWbrhWAResOHe...,LIL Milan,2024-09-07 13:06:24,Riuscito,20.0,0.0,20.0,2024-09-07 13:06:04,FALSO,340.0
43,#41675,defrancesco-mario@libero.it,paid,2024-09-11 18:48:47 +0200,fulfilled,2024-09-12 18:19:50 +0200,no,EUR,320.0,0.0,57.72,320.0,NaN,0.0,NaN,2024-09-11 18:48:47 +0200,1,Giotto Ring - White / 11,80.0,NaN,015790000156,True,True,pending,Mario De Francesco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IT,NaN,NaN,NaN,NaN,Qromo,rfzxzpLXNAh2KaOYYvozHLLc4,0.0,LIL Milan,160.0,Veronica Varetta,LIL House,22,6300088107357,<NA>,Low,pos,0.0,IT IVA 22%,57.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2491,NaN,NaN,NaN,rfzxzpLXNAh2KaOYYvozHLLc4,NaN,<NA>,rfzxzpLXNAh2KaOYYvozHLLc4,LIL Milan,2024-09-11 18:48:47,Riuscito,160.0,0.0,160.0,2024-09-11 18:48:29,FALSO,NaN
44,#41675,defrancesco-mario@libero.it,paid,2024-09-11 18:48:47 +0200,fulfilled,2024-09-12 18:19:50 +0200,no,EUR,320.0,0.0,57.72,320.0,NaN,0.0,NaN,2024-09-11 18:48:47 +0200,1,Giotto Ring - White / 12,80.0,0.0,015790000157,True,True,fulfilled,Mario De Francesco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IT,NaN,NaN,NaN,NaN,Qromo,rfzxzpLXNAh2KaOYYvozHLLc4,0.0,LIL Milan,160.0,Veronica Varetta,LIL House,22,6300088107357,<NA>,Low,pos,0.0,IT IVA 22%,57.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2491,NaN,NaN,NaN,rfzxzpLXNAh2KaOYYvozHLLc4,NaN,<NA>,rfzxzpLXNAh2KaOYYvozHLLc4,LIL Milan,2024-09-11 18:48:47,Riuscito,160.0,0.0,160.0,2024-09-11 18:48:29,FALSO,NaN
45,#41675,defrancesco-mario@libero.it,paid,2024-09-11 18:48:47 +0200,fulfilled,2024-09-12 18:19:50 +0200,no,EUR,320.0,0.0,57.72,320.0,NaN,0.0,NaN,2024-09-11 18:48:47 +0200,1,Giotto Ring - Yellow / 12,80.0,0.0,015790000146,True,True,pending,Mario De Francesco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IT,NaN,NaN,NaN,NaN,Qromo,rfzxzpLXNAh2KaOYYvozHLLc4,0.0,LIL Milan,160.0,Veronica Varetta,LIL House,22,6300088107357,<NA>,Low,pos,0.0,IT IVA 22%,57.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22-2491,NaN,NaN,NaN,rfzxzpLXNAh2KaOYYvozHLLc4,NaN,<NA>,rfzxzpLXNAh2KaOYYvozHLLc4,LIL Milan,2024-09-11 18:48:47,Riuscito,160.0,0.0,160.0,2024-09-11 18:48:29,FALSO,NaN


'LIL Milan'

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,Brand,Paid_datetime,Data,Operazione,Dettagli,Conto o carta,Contabilizzazione,Categoria,Valuta,Importo,Data_datetime,CHECK,CHECK_VALORE_GIFT_CARD,Days_difference
5,#41965,isabellaclb@yahoo.it,pending,NaN,unfulfilled,NaN,yes,EUR,160.0,20.0,32.46,180.0,NaN,0.0,UPS Express Shipping,2024-09-25 13:01:09 +0200,2.0,Giotto Ring - Yellow / 7,80.0,NaN,015790000141,True,True,pending,Isabella Oriani,"Via Superiore, 5B, Lierna, LC, Italia","Via Superiore, 5B, Lierna, LC, Italia",NaN,NaN,Lierna,'23827,LC,IT,+393405948172,Isabella Oriani,"Via Superiore, 5B, Lierna, LC, Italia","Via Superiore, 5B, Lierna, LC, Italia",NaN,NaN,Lierna,'23827,LC,IT,+393405948172,NaN,lang: it\nInvoice Language: it\nDo you need ou...,NaN,Bonifico,NaN,0.0,LIL Milan,180.0,NaN,Firgun House,<NA>,6320453452125,<NA>,Low,web,0.0,IT IVA 22%,32.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lecco,Lecco,ryrWHBRbo98sPlJRXbd5557PZ,NaN,<NA>,ryrWHBRbo98sPlJRXbd5557PZ,LIL Milan,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NON TROVATO,NaN,NaN


In [4]:
#ordini
ordini_processor = Ordini(filepath, mese = 10)
ordini = ordini_processor.preprocess()

#run matchers
shopify_matcher = ShopifyMatcher(filepath, df_ordini=ordini)
scalapay_matcher = ScalapayMatcher(filepath, df_ordini=ordini)
satispay_matcher = SatispayMatcher(filepath, df_ordini=ordini)
paypal_matcher = PaypalMatcher(filepath, df_ordini=ordini)
qromo_matcher = QromoMatcher(filepath, df_ordini=ordini)
bonifico_matcher = BonificoMatcher(filepath, df_ordini=ordini)

# Create the matchers list
matchers = [
    shopify_matcher,
    scalapay_matcher,
    satispay_matcher,
    paypal_matcher,
    qromo_matcher,
    bonifico_matcher
]

#excel
runner = MatcherRunner(matchers, ordini)
df_check_all, df_ordini_all, df = runner.run_all_matchers(mese=10, anno= 2024)

Non ci sono pagamenti con Satispay
Non ci sono pagamenti col POS
Non ci sono bonifici
All 'Names' have consistent 'Total' values.



'LIL Milan'

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,Brand,Order,Amount,CHECK,CHECK_VALORE_GIFT_CARD
1,#42097,francesca.tarantino7@gmail.com,paid,2024-10-01 09:30:00 +0200,fulfilled,2024-10-12 14:07:53 +0200,yes,EUR,216.0,0.0,38.96,96.0,LILGIRL,44.0,Firgun House,2024-10-01 09:30:00 +0200,0.0,Boys Tears Ring - Yellow / 12,120.0,NaN,015790001402,True,True,pending,Greta Tarantino,"Corso di Porta Romana 95, Citofono Tarantino/C...",Corso di Porta Romana 95,Citofono Tarantino/Carniglia,NaN,Milano,'20122,MI,IT,+393397801592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IT,NaN,NaN,lang: it\nInvoice Language: it\nDo you need ou...,NaN,Shopify Payments,rr1x7bkqrzUIJ28vUULEpEDeJ,0.0,LIL Milan,108.0,NaN,Firgun House,<NA>,6328947048797,<NA>,Low,web,0.0,IT IVA 22%,38.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Milan,NaN,rr1x7bkqrzUIJ28vUULEpEDeJ,NaN,<NA>,rr1x7bkqrzUIJ28vUULEpEDeJ,LIL Milan,#42097,108.0,FALSO,NaN
2,#42097,francesca.tarantino7@gmail.com,paid,2024-10-01 09:30:00 +0200,fulfilled,2024-10-12 14:07:53 +0200,yes,EUR,216.0,0.0,38.96,96.0,LILGIRL,44.0,Firgun House,2024-10-01 09:30:00 +0200,1.0,Boys Tears Ring - Yellow / 15,140.0,0.0,015790001403,True,True,fulfilled,Greta Tarantino,"Corso di Porta Romana 95, Citofono Tarantino/C...",Corso di Porta Romana 95,Citofono Tarantino/Carniglia,NaN,Milano,'20122,MI,IT,+393397801592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IT,NaN,NaN,lang: it\nInvoice Language: it\nDo you need ou...,NaN,Shopify Payments,rr1x7bkqrzUIJ28vUULEpEDeJ,0.0,LIL Milan,108.0,NaN,Firgun House,<NA>,6328947048797,<NA>,Low,web,32.0,IT IVA 22%,38.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Milan,NaN,rr1x7bkqrzUIJ28vUULEpEDeJ,NaN,<NA>,rr1x7bkqrzUIJ28vUULEpEDeJ,LIL Milan,#42097,108.0,FALSO,NaN


'LIL Milan'

,Name,Email,Financial Status,Paid at,Fulfillment Status,Fulfilled at,Accepts Marketing,Currency,Subtotal,Shipping,Taxes,Total,Discount Code,Discount Amount,Shipping Method,Created at,Lineitem quantity,Lineitem name,Lineitem price,Lineitem compare at price,Lineitem sku,Lineitem requires shipping,Lineitem taxable,Lineitem fulfillment status,Billing Name,Billing Street,Billing Address1,Billing Address2,Billing Company,Billing City,Billing Zip,Billing Province,Billing Country,Billing Phone,Shipping Name,Shipping Street,Shipping Address1,Shipping Address2,Shipping Company,Shipping City,Shipping Zip,Shipping Province,Shipping Country,Shipping Phone,Notes,Note Attributes,Cancelled at,Payment Method,Payment Reference,Refunded Amount,Vendor,Outstanding Balance,Employee,Location,Device ID,Id,Tags,Risk Level,Source,Lineitem discount,Tax 1 Name,Tax 1 Value,Tax 2 Name,Tax 2 Value,Tax 3 Name,Tax 3 Value,Tax 4 Name,Tax 4 Value,Tax 5 Name,Tax 5 Value,Phone,Receipt Number,Duties,Billing Province Name,Shipping Province Name,Payment ID,Payment Terms Name,Next Payment Due At,Payment References,Brand,Matched Reference,N° ordine commerciante,Lordo,Valuta,CHECK,CHECK_VALORE_GIFT_CARD
131,#42424,gioiaa.taddeoo@gmail.com,paid,2024-10-17 18:12:27 +0200,unfulfilled,NaN,yes,EUR,180.0,0.0,32.46,180.0,NaN,0.0,Ups Standard Shipping,2024-10-17 18:12:26 +0200,1.0,Primo bacio Necklace - Yellow / 38cm,180.0,NaN,015790001025,True,True,pending,Anna Balducci,Via del Cigno 59,Via del Cigno 59,NaN,NaN,Rimini,'47923,RN,IT,+393397358394,Anna Balducci,Via del Cigno 59,Via del Cigno 59,NaN,NaN,Rimini,'47923,RN,IT,+393397358394,NaN,lang: it\nInvoice Language: it\nDo you need ou...,NaN,PayPal Express Checkout,rAG3aVgotJvs5Abjs4NNhljTf,0.0,LIL Milan,0.0,NaN,Firgun House,<NA>,6354953732445,<NA>,Low,web,0.0,IT IVA 22%,32.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rimini,Rimini,rAG3aVgotJvs5Abjs4NNhljTf,NaN,<NA>,rAG3aVgotJvs5Abjs4NNhljTf,LIL Milan,NaN,NaN,NaN,NaN,NON TROVATO,NaN
